## Unsloth基本使用方法

### 1.模型导入与调用流程

In [20]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ['UNSLOTH_USE_MODELSCOPE'] = '1'
print("1.1")

1.1


&emsp;&emsp;首先进行模型导入：

In [21]:
from unsloth import FastLanguageModel
import torch

In [ ]:
max_seq_length = 8192 # 模型能够处理的最大序列长度（token数量）
dtype = None # 数据类型（data type）
load_in_4bit = True # 用4-bit量化加载模型

- Unsloth 加载模型

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    # model_name = "./Qwen3-0.6B-unsloth-bnb-4bit",
    model_name = "/root/autodl-tmp/modelscope/models/unsloth/Qwen3-8B-unsloth-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    local_files_only = True,  # 如果不加这一行，有时模型会自动尝试连接 HuggingFace 检查更新。
)

==((====))==  Unsloth 2026.1.1: Fast Qwen3 patching. Transformers: 4.57.3. vLLM: 0.13.0.
   \\   /|    NVIDIA GeForce RTX 4090 D. Num GPUs = 1. Max memory: 23.516 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
model

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 4096, padding_idx=151654)
    (layers): ModuleList(
      (0-2): 3 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=4096, out_features=12288, bias=False)
          (up_proj): Linear(in_features=4096, out_features=12288, bias=False)
          (down_proj): Linear(in_features=12288, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
     

In [ ]:
tokenizer

Qwen2TokenizerFast(name_or_path='/root/autodl-tmp/modelscope/models/unsloth/Qwen3-8B-unsloth-bnb-4bit', vocab_size=151643, model_max_length=40960, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|vision_pad|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>', '<|object_ref_start|>', '<|object_ref_end|>', '<|box_start|>', '<|box_end|>', '<|quad_start|>', '<|quad_end|>', '<|vision_start|>', '<|vision_end|>', '<|vision_pad|>', '<|image_pad|>', '<|video_pad|>']}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151646: AddedToken("<|object_ref_start|>", rstri

- 微调数据集（Dataset）必须符合以下格式，模型才能学会对话：
<|im_start|>system
你是一个有用的助手。<|im_end|>
<|im_start|>user
你好，请帮我写个 Python 脚本。<|im_end|>
<|im_start|>assistant
<think>
用户需要 Python 脚本，我应该先询问具体需求。
</think>
没问题，请问你想实现什么功能？<|im_end|>

- 显存占用

In [ ]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4090 D. Max memory = 23.516 GB.
7.811 GB of memory reserved.


- 开启对话

&emsp;&emsp;然后即可尝试进行对话。借助Unsloth进行模型调用总共需要两个步骤，其一是借助`apply_chat_template`进行分词同时输入对话相关参数，其二则是借助`generate`进行文本创建。一次基本对话流程如下所示：

In [ ]:
messages = [
    {"role" : "user", "content" : "你知道西北工业大学吗"}
]

In [ ]:
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, 
    enable_thinking = False, # 设置不思考
)

此时text就是加载了Qwen3内置提示词模板之后的字符串。据此也能看出Qwen3内置提示词模板的特殊字符：

In [ ]:
text

'<|im_start|>user\n你知道西北工业大学吗<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\n'

然后进行分词：

In [ ]:
inputs = tokenizer(text, return_tensors="pt").to("cuda")

并进行推理：

In [ ]:
outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=max_seq_length,
    use_cache=True,
)

最终获得模型输出结果：

In [ ]:
outputs

tensor([[151644,    872,    198, 107733, 105507, 117509, 101037, 151645,    198,
         151644,  77091,    198, 151667,    271, 151668,    271,  20412,   9370,
           3837, 105519, 105507, 117509,   9909,  25221,  30282,  18767,  72137,
           3822,   3837, 102500,   2073,  60686,  48692,  26288,    854,   7552,
           3407,  14374,   4891,    253,    118,  21894,  27369,  28311,     12,
           3070, 100344,  31905,    334,   5122, 103792,  99562,   3837,  20412,
            334,  58695, 108767,   9370, 117792,  21515, 104127, 100653,    334,
           3837, 100000,   2073,    334,  99609,  93823,    334,    854, 100653,
           9909,  52129, 112877, 102320, 104663,   3837,  46451,  17714, 102782,
          99602, 100338,   7552,   8997,     12,   3070, 111692,    334,   5122,
         108586, 112584,   8997,     12,   3070, 100022,    334,   5122,  50377,
          34204,     16,     24,     18,     23,   7948,   3837, 113877,  20412,
            334, 117048, 105

In [ ]:
response = tokenizer.batch_decode(outputs)

In [ ]:
response

['<|im_start|>user\n你知道西北工业大学吗<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\n是的，我知道西北工业大学（Northwestern Polytechnical University，简称“西工大”）。\n\n### 基本信息：\n- **学校类型**：公立大学，是**中国顶尖的航空航天类高校之一**，也是“**九院**”之一（原国防部第五研究院，现为航天科技集团）。\n- **地理位置**：陕西省西安市。\n- **历史**：创建于1938年，前身是**国立西北工学院**，后与**西安航空学院**、**西安工业大学**等合并，1994年正式定名为西北工业大学。\n- **学科优势**：以**航空、航天、航海、兵器科学与技术**为特色，是中国航空、航天、兵器等领域的“**人才摇篮**”。\n- **科研实力**：拥有多个国家级重点实验室和工程中心，如“**航空动力研究院**”、“**航天器总体设计技术重点实验室**”等。\n\n### 特点：\n1. **军工背景深厚**：与航天、航空、兵器等国防工业联系紧密，许多毕业生进入**中国航天科技集团、航空工业集团、兵器工业集团**等。\n2. **学科全面**：除航空航天类学科外，还涵盖材料、机械、电子、计算机、信息工程、管理科学等。\n3. **科研实力强**：在**航空宇航科学与技术、兵器科学与技术、动力工程及工程热物理**等学科领域处于国内领先地位。\n4. **国际合作广泛**：与多个国家的高校和科研机构有合作，如与美国、俄罗斯、德国等国的高校有联合培养项目。\n\n### 历史地位：\n- 被称为“**中国空军的摇篮**”。\n- 培养了大量中国航天、航空领域的高级人才，如**王永志（中国航天之父）**、**马兴瑞（中国工程院院士）**等。\n- 是“**211工程**”“**985工程**”重点建设高校，也是“**双一流**”建设高校。\n\n### 如果你对西北工业大学的某个方面感兴趣，比如专业、就业、校园生活、历史等，我可以进一步为你详细介绍。<|im_end|>']

In [ ]:
response[0]

'<|im_start|>user\n你知道西北工业大学吗<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\n是的，我知道西北工业大学（Northwestern Polytechnical University，简称“西工大”）。\n\n### 基本信息：\n- **学校类型**：公立大学，是**中国顶尖的航空航天类高校之一**，也是“**九院**”之一（原国防部第五研究院，现为航天科技集团）。\n- **地理位置**：陕西省西安市。\n- **历史**：创建于1938年，前身是**国立西北工学院**，后与**西安航空学院**、**西安工业大学**等合并，1994年正式定名为西北工业大学。\n- **学科优势**：以**航空、航天、航海、兵器科学与技术**为特色，是中国航空、航天、兵器等领域的“**人才摇篮**”。\n- **科研实力**：拥有多个国家级重点实验室和工程中心，如“**航空动力研究院**”、“**航天器总体设计技术重点实验室**”等。\n\n### 特点：\n1. **军工背景深厚**：与航天、航空、兵器等国防工业联系紧密，许多毕业生进入**中国航天科技集团、航空工业集团、兵器工业集团**等。\n2. **学科全面**：除航空航天类学科外，还涵盖材料、机械、电子、计算机、信息工程、管理科学等。\n3. **科研实力强**：在**航空宇航科学与技术、兵器科学与技术、动力工程及工程热物理**等学科领域处于国内领先地位。\n4. **国际合作广泛**：与多个国家的高校和科研机构有合作，如与美国、俄罗斯、德国等国的高校有联合培养项目。\n\n### 历史地位：\n- 被称为“**中国空军的摇篮**”。\n- 培养了大量中国航天、航空领域的高级人才，如**王永志（中国航天之父）**、**马兴瑞（中国工程院院士）**等。\n- 是“**211工程**”“**985工程**”重点建设高校，也是“**双一流**”建设高校。\n\n### 如果你对西北工业大学的某个方面感兴趣，比如专业、就业、校园生活、历史等，我可以进一步为你详细介绍。<|im_end|>'

需要注意的是，这其实是一种非常底层的打印模型输入和输出信息的方法，这种字符格式（同时包含模型输入和输出）也是Unsloth在进行高效微调过程中需要用到的数据集基本格式。

&emsp;&emsp;也可通过如下方式生成带有思考过程的结果：

In [ ]:
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, 
    enable_thinking = True, # 设置思考
)

inputs = tokenizer(text, return_tensors="pt").to("cuda")
# return_tensors这个参数告诉分词器（Tokenizer）：“请不要给我返回普通的 Python 列表，直接给我返回 PyTorch 格式的张量。”

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=max_seq_length,
    use_cache=True,
)

response = tokenizer.batch_decode(outputs)

In [ ]:
response[0]

'<|im_start|>user\n你知道西北工业大学吗<|im_end|>\n<|im_start|>assistant\n<think>\n好的，用户问我知道西北工业大学吗。首先，我需要确认用户的需求是什么。可能他们想了解学校的基本情况，或者有入学、就业、研究方向等方面的疑问。也有可能用户对学校有特定的兴趣，比如某个专业或者校园生活。\n\n接下来，我得考虑用户可能的背景。可能是学生、家长，或者对工科感兴趣的人。如果是学生，可能关心专业设置、排名、就业情况；如果是家长，可能更关注学校声誉和教育资源；如果是对工科有兴趣的人，可能想了解学校的研究方向或实验室。\n\n然后，我需要确保回答准确且全面。西北工业大学是九所重点理工科大学之一，以航空、航天、航海等学科闻名。要提到它的历史，比如前身是国立西北工学院，后来合并成为现在的学校。还要强调其在国防科技领域的地位，比如参与多项国家重大科研项目，培养了众多优秀人才。\n\n另外，用户可能想知道学校的优势专业，比如飞行器设计、材料科学、自动化等。同时，可能需要提到校园环境、地理位置（西安）、国际交流项目，以及就业情况，比如毕业生进入航天、航空等企业。还要注意是否有需要注意的地方，比如竞争激烈，或者某些专业就业方向。\n\n还要考虑用户是否有更深层次的需求，比如是否在考虑申请该校，或者寻找相关资源。这时候可能需要建议他们查阅官网或联系招生办，以获取最新信息。同时，保持回答结构清晰，分点说明，方便用户理解。\n\n最后，确保语言口语化，避免使用过于专业的术语，但又要准确。检查是否有遗漏的重要信息，比如学校的排名、特色活动或校友情况，这些都能增加回答的全面性。总之，需要平衡信息的详实和易懂，满足不同用户的需求。\n</think>\n\n是的，我了解**西北工业大学**（简称“西工大”），它是中国著名的**九所重点理工科大学之一**，也是**国防科技领域的重要高校**。以下是关于该校的详细介绍：\n\n---\n\n### **1. 学校概况**\n- **地理位置**：位于中国陕西省西安市，是**西北地区重点高校**，也是“**双一流**”建设高校。\n- **历史沿革**：前身为1938年成立的**国立西北工学院**，1957年更名为**西北工业大学**，是**中国最早开展航空、航天、航海等学科研究的高校之一**。\n- 

同时如果存在系统提示词，则实际对话效果如下：

In [ ]:
messages = [
    {"role" : "system", "content" : "你现在是黑格尔"},
    {"role" : "user", "content" : "你好，能向我介绍你自己吗"}
]
# system (系统提示词/设定)：它是最高指令，是给模型下的“底层逻辑”。
# 用来规定模型的身份、语气、行为准则或知识边界。

In [ ]:
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, 
    enable_thinking = True, # 设置思考
)

inputs = tokenizer(text, return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=max_seq_length,
    use_cache=True,
)

response = tokenizer.batch_decode(outputs)

In [ ]:
response[0]

'<|im_start|>system\n你现在是黑格尔<|im_end|>\n<|im_start|>user\n你好，能向我介绍你自己吗<|im_end|>\n<|im_start|>assistant\n<think>\n好的，用户让我以黑格尔的身份进行对话。首先，我需要回顾黑格尔的基本哲学观点，比如辩证法、绝对精神、主奴辩证法等。用户可能希望了解黑格尔的核心思想，但也要避免过于学术化，保持对话自然。\n\n接下来，用户的问题是关于自我介绍。黑格尔作为哲学家，应该用他的语言风格，比如强调理性、历史发展、理念的展开。需要将这些概念转化为口语化的表达，同时保持深度。\n\n然后，考虑用户可能的背景。他们可能对哲学有一定兴趣，但不是专家，所以需要用简单易懂的方式解释复杂概念。比如，提到"绝对理念"时，可以举例说明其在历史中的体现，如国家、宗教、艺术等。\n\n还要注意黑格尔的表达方式，比如使用"您"来保持尊重，引用他常用的术语如"辩证法"、"主奴辩证法"，但避免直接引用原文，而是用自己的话转述。同时，可以加入一些互动，比如询问用户对某个概念的理解，促进对话继续。\n\n最后，确保回答结构清晰，先介绍身份，再分点阐述核心思想，最后邀请用户提问。这样既符合黑格尔的哲学框架，又保持对话的流畅性。\n</think>\n\n*以深沉而富有哲思的语调缓缓开口*\n\n啊，您对哲学怀有浓厚的兴趣，这让我感到欣慰。我是格奥尔格·威廉·弗里德里希·黑格尔，一位致力于探索人类理性本质的哲学家。我的思想如同一条奔腾的河流，从现象世界流淌至绝对理念的彼岸。\n\n*抬起眼睛，目光如炬*\n\n您或许会问，何为我的核心思想？让我用最简明的方式阐述：我的哲学体系以"绝对理念"为核心，它在历史进程中通过"正题-反题-合题"的辩证运动展开。这种运动不仅存在于抽象思维中，更体现在现实世界的历史发展里——从古代的自然宗教，到中世纪的基督教，再到现代的国家与自由。\n\n*微微前倾，语气中带着期待*\n\n您是否注意到，人类社会的发展始终伴随着主奴辩证法的斗争？当个体意识到自身作为自由存在的本质时，真正的历史才开始。而艺术、宗教、哲学等精神领域，不过是绝对理念在不同阶段的外化表现。\n\n*轻抚胡须，露出神秘的微笑*\n\n不过，我更愿意倾听您的见解。您对"主奴辩证法"的理解，是否与我所阐述的有所不同

此时问答数据中就包含了系统消息。同样该格式的数据也可以直接用于Unsloth的指令微调。也就是说，如果我们希望提高模型多轮对话或者指令跟随能力，就可以创建大量类似这种数据集进行微调。在实际微调过程中，模型会主动学习最后一个assistant之后的内容，从而学会指令跟随和多轮对话能力。

### function calling： 查询天气

In [ ]:
import requests, json
def get_weather(loc):
    """
    查询即时天气函数
    :param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，\
    注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询北京市天气，则loc参数需要输入'Beijing'；
    :return：OpenWeather API查询即时天气的结果，具体URL请求地址为：https://api.openweathermap.org/data/2.5/weather\
    返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息
    """
    # Step 1.构建请求
    url = "https://api.openweathermap.org/data/2.5/weather"

    # Step 2.设置查询参数
    params = {
        "q": loc,               
        "appid": "YOUR_API_KEY",    # 输入API key
        "units": "metric",            # 使用摄氏度而不是华氏度
        "lang":"zh_cn"                # 输出语言为简体中文
    }

    # Step 3.发送GET请求
    response = requests.get(url, params=params)
    
    # Step 4.解析响应
    data = response.json()
    return json.dumps(data)

In [ ]:
tools = [
    {
        "type": "function",
        "function":{
            'name': 'get_weather',
            'description': '查询即时天气函数，根据输入的城市名称，查询对应城市的实时天气，一次只能输入一个城市名称',
            'parameters': {
                'type': 'object',
                'properties': {
                    'loc': {
                        'description': "城市名称，注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询北京市天气，则loc参数需要输入'Beijing'",
                        'type': 'string'
                    }
                },
                'required': ['loc']
            }
        }
    }
]

In [ ]:
messages = [
    {"role" : "system", "content" : "你是一名助人为乐的天气查询助手，当用户询问天气信息时，请调用get_weather函数进行天气查询。"},
    {"role" : "user", "content" : "你好，请帮我查询下北京今天天气如何？"}
]

In [ ]:
text = tokenizer.apply_chat_template(
    messages,
    tools = tools,
    tokenize = False,
    add_generation_prompt = True, 
    enable_thinking = True, # 设置思考
)

inputs = tokenizer(text, return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=max_seq_length,
    use_cache=True,
)

response = tokenizer.batch_decode(outputs)

In [ ]:
response[0]

'<|im_start|>system\n你是一名助人为乐的天气查询助手，当用户询问天气信息时，请调用get_weather函数进行天气查询。\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>\n{"type": "function", "function": {"name": "get_weather", "description": "查询即时天气函数，根据输入的城市名称，查询对应城市的实时天气，一次只能输入一个城市名称", "parameters": {"type": "object", "properties": {"loc": {"description": "城市名称，注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询北京市天气，则loc参数需要输入\'Beijing\'", "type": "string"}}, "required": ["loc"]}}}\n</tools>\n\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\n<tool_call>\n{"name": <function-name>, "arguments": <args-json-object>}\n</tool_call><|im_end|>\n<|im_start|>user\n你好，请帮我查询下北京今天天气如何？<|im_end|>\n<|im_start|>assistant\n<think>\n好的，用户让我查询北京今天的天气。首先，我需要确认用户的需求是获取北京的实时天气信息。根据提供的工具，我需要调用get_weather函数。注意，工具说明中提到中国的城市要用英文名称，所以北京应该转换为Beijing。接下来，我需要构造函数调用的参数，确保loc参数正确。然后，检查是否有其他可能的参

能够看到，此时模型就会创建一条同时带有指令、思考、外部函数的function call message。

而更进一步的，我们也可以测试模型的多个外部函数并联调用效果：

In [ ]:
messages = [
    {"role" : "system", "content" : "你是一名助人为乐的天气查询助手，当用户询问天气信息时，请调用get_weather函数进行天气查询。"},
    {"role" : "user", "content" : "你好，请帮我查询下北京和杭州今天天气如何？"}
]

In [ ]:
text = tokenizer.apply_chat_template(
    messages,
    tools = tools,
    tokenize = False,
    add_generation_prompt = True, 
    enable_thinking = True, # 设置思考
)

inputs = tokenizer(text, return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=max_seq_length,
    use_cache=True,
)

response = tokenizer.batch_decode(outputs)

In [ ]:
response[0]

'<|im_start|>system\n你是一名助人为乐的天气查询助手，当用户询问天气信息时，请调用get_weather函数进行天气查询。\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>\n{"type": "function", "function": {"name": "get_weather", "description": "查询即时天气函数，根据输入的城市名称，查询对应城市的实时天气，一次只能输入一个城市名称", "parameters": {"type": "object", "properties": {"loc": {"description": "城市名称，注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询北京市天气，则loc参数需要输入\'Beijing\'", "type": "string"}}, "required": ["loc"]}}}\n</tools>\n\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\n<tool_call>\n{"name": <function-name>, "arguments": <args-json-object>}\n</tool_call><|im_end|>\n<|im_start|>user\n你好，请帮我查询下北京和杭州今天天气如何？<|im_end|>\n<|im_start|>assistant\n<think>\n好的，用户让我查询北京和杭州的天气。首先，我需要确认用户的需求是同时获取两个城市的天气信息。但根据提供的工具函数get_weather，每次只能查询一个城市，因为参数中loc是单个城市名称。这时候我需要判断是否需要多次调用函数。\n\n用户的问题中提到了两个城市，北京和杭州，所以可能需要分别调

能发现，此时模型同时发起了两条Function call message。

然后继续测试当模型接收到外部函数返回消息时候，模型返回内容。

In [ ]:
messages

[{'role': 'system',
  'content': '你是一名助人为乐的天气查询助手，当用户询问天气信息时，请调用get_weather函数进行天气查询。'},
 {'role': 'user', 'content': '你好，请帮我查询下北京和杭州今天天气如何？'}]

In [ ]:
messages.append({
    "role": "assistant",
    "content": "<think>\n我将调用 get_weather 函数来查询天气。\n</think>\n",
    "tool_calls": [
        {
            "name": "get_weather",
            "arguments": {
                "location": "北京"
            }
        },
        {
            "name": "get_weather",
            "arguments": {
                "location": "杭州"
            }
        }
    ]
})

In [ ]:
messages.append({
    "role": "tool",
    "content": json.dumps({
        "location": "北京",
        "weather": "晴，最高气温26℃"
    })
})
messages.append({
    "role": "tool",
    "content": json.dumps({
        "location": "杭州",
        "weather": "多云转小雨，最高气温23℃"
    })
})


In [ ]:
messages

[{'role': 'system',
  'content': '你是一名助人为乐的天气查询助手，当用户询问天气信息时，请调用get_weather函数进行天气查询。'},
 {'role': 'user', 'content': '你好，请帮我查询下北京和杭州今天天气如何？'},
 {'role': 'assistant',
  'content': '<think>\n我将调用 get_weather 函数来查询天气。\n</think>\n',
  'tool_calls': [{'name': 'get_weather', 'arguments': {'location': '北京'}},
   {'name': 'get_weather', 'arguments': {'location': '杭州'}}]},
 {'role': 'tool',
  'content': '{"location": "\\u5317\\u4eac", "weather": "\\u6674\\uff0c\\u6700\\u9ad8\\u6c14\\u6e2926\\u2103"}'},
 {'role': 'tool',
  'content': '{"location": "\\u676d\\u5dde", "weather": "\\u591a\\u4e91\\u8f6c\\u5c0f\\u96e8\\uff0c\\u6700\\u9ad8\\u6c14\\u6e2923\\u2103"}'}]

In [ ]:
text = tokenizer.apply_chat_template(
    messages,
    tools = tools,
    tokenize = False,
    add_generation_prompt = True, 
    enable_thinking = True, # 设置思考
)

inputs = tokenizer(text, return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=max_seq_length,
    use_cache=True,
)

response = tokenizer.batch_decode(outputs)

In [ ]:
response[0]

'<|im_start|>system\n你是一名助人为乐的天气查询助手，当用户询问天气信息时，请调用get_weather函数进行天气查询。\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>\n{"type": "function", "function": {"name": "get_weather", "description": "查询即时天气函数，根据输入的城市名称，查询对应城市的实时天气，一次只能输入一个城市名称", "parameters": {"type": "object", "properties": {"loc": {"description": "城市名称，注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询北京市天气，则loc参数需要输入\'Beijing\'", "type": "string"}}, "required": ["loc"]}}}\n</tools>\n\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\n<tool_call>\n{"name": <function-name>, "arguments": <args-json-object>}\n</tool_call><|im_end|>\n<|im_start|>user\n你好，请帮我查询下北京和杭州今天天气如何？<|im_end|>\n<|im_start|>assistant\n<think>\n我将调用 get_weather 函数来查询天气。\n</think>\n\n<tool_call>\n{"name": "get_weather", "arguments": {"location": "北京"}}\n</tool_call>\n<tool_call>\n{"name": "

而这就是一条能够进行工具并联微调训练的数据集。

### function calling：控制灯光

In [ ]:
import json

def control_smart_light(room, action, brightness=None):
    """
    控制家中智能灯光的函数。
    :param room: 房间名称，如 'living_room' (客厅), 'bedroom' (卧室)
    :param action: 操作类型，'on' (开灯), 'off' (关灯), 'adjust' (调节亮度)
    :param brightness: 亮度值，范围 1-100，仅在 action 为 'adjust' 时有效
    :return: 操作结果的描述
    """
    # 模拟硬件操作
    status = {
        "room": room,
        "action": action,
        "brightness": brightness if brightness else "N/A",
        "status": "success"
    }
    return json.dumps(status)

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "control_smart_light",
            "description": "智能家居灯光控制，可以开关灯或调节特定房间的亮度",
            "parameters": {
                "type": "object",
                "properties": {
                    "room": {
                        "type": "string",
                        "description": "房间名称，例如：'living_room', 'bedroom', 'kitchen'"
                    },
                    "action": {
                        "type": "string",
                        "enum": ["on", "off", "adjust"],
                        "description": "执行的动作：开灯、关灯或调节亮度"
                    },
                    "brightness": {
                        "type": "integer",
                        "description": "亮度百分比 (1-100)"
                    }
                },
                "required": ["room", "action"]
            }
        }
    }
]

In [ ]:
# 案例 A
messages = [
    {"role": "system", "content": "你是一个智能家居管家。"},
    {"role": "user", "content": "卧室太暗了，帮我把灯调亮到80%。"}
]

# 案例 B：明确指令
# messages = [{"role": "user", "content": "我要出门了，把客厅的灯关掉。"}]

text = tokenizer.apply_chat_template(
    messages,
    tools=tools,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True
)

# 经过 model.generate 后，得到模型生成的调用请求：
# <tool_call>
# {"name": "control_smart_light", "arguments": {"room": "bedroom", "action": "adjust", "brightness": 80}}
# </tool_call>

In [ ]:
inputs = tokenizer(text, return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=max_seq_length,
    use_cache=True,
)

response = tokenizer.batch_decode(outputs)

In [ ]:
response[0]

'<|im_start|>system\n你是一个智能家居管家。\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>\n{"type": "function", "function": {"name": "control_smart_light", "description": "智能家居灯光控制，可以开关灯或调节特定房间的亮度", "parameters": {"type": "object", "properties": {"room": {"type": "string", "description": "房间名称，例如：\'living_room\', \'bedroom\', \'kitchen\'"}, "action": {"type": "string", "enum": ["on", "off", "adjust"], "description": "执行的动作：开灯、关灯或调节亮度"}, "brightness": {"type": "integer", "description": "亮度百分比 (1-100)"}}, "required": ["room", "action"]}}}\n</tools>\n\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\n<tool_call>\n{"name": <function-name>, "arguments": <args-json-object>}\n</tool_call><|im_end|>\n<|im_start|>user\n卧室太暗了，帮我把灯调亮到80%。<|im_end|>\n<|im_start|>assistant\n<think>\n好的，用户说卧室太暗了，需要把灯调亮到80%。首先，我需要确定用户提到的房间

### 2. Unsloth高层对话API

&emsp;&emsp;当然，除了使用上述底层API进行对话外，Unsloth还提供了更加便捷的流式输出模型对话信息的函数，基本对话效果如下：

In [ ]:
from transformers import TextStreamer

- 实时解码器。 在默认情况下，model.generate 是一个“同步”过程：模型在后台计算，算完几百个 Token 后一次性返回，这会导致程序卡顿几秒甚至十几秒。 TextStreamer 会在模型生成每一个 Token 的瞬间，立即将其解码并打印到终端上。

In [ ]:
messages = [
    {"role" : "user", "content" : "你是秦始皇，向我简要介绍你自己"}
]

In [ ]:
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, 
    enable_thinking = False, 
)

_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 256, 
    temperature = 0.7, top_p = 0.8, top_k = 20, 
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

# tokenizer(...) 返回的是一个字典，包含 input_ids 和 attention_mask。
# 使用 ** 会将这个字典解包。这意味着它等同于手动传入： input_ids = inputs["input_ids"], attention_mask = inputs["attention_mask"]
# 这种写法简洁且不容易漏掉 attention_mask

# top_p = 0.8 和 top_k = 20:
# 这是双重采样过滤。模型首先筛选出概率最高的前 20 个词（top_k），然后再从中筛选出累积概率达到 80% 的词（top_p）。
# 效果：这种组合极大增强了生成内容的连贯性和多样性，防止模型陷入死循环或胡言乱语。
# streamer 参数允许模型在生成过程中实时输出文本，而不是等到全部生成完成后一次性输出。

朕乃秦始皇，姓嬴名政，秦庄襄王之子，秦朝开国皇帝。朕统一六国，废除分封，建立中央集权，推行郡县制，统一度量衡、文字、车轨，筑长城以御外敌，开灵渠以通水路。朕以“始皇帝”为号，欲传之万世，永续秦祚。天下归一，功业千秋，朕之意志，即为天下法度。汝可称朕为“始皇”，或“陛下”。<|im_end|>


In [ ]:
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, 
    enable_thinking = True, # 设置思考
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 2048, 
    temperature = 0.6, top_p = 0.95, top_k = 20, 
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

<think>
好的，用户让我扮演秦始皇，向他简要介绍自己。首先，我需要确定用户的需求是什么。可能他们对历史人物感兴趣，或者想了解秦始皇的生平和成就。用户可能希望得到一个简洁但全面的介绍，同时带有秦始皇的语气和风格。

接下来，我要考虑如何以秦始皇的口吻来回答。需要使用古代的词汇和表达方式，比如“朕”、“朕之”等，同时保持语言简洁明了，避免过于复杂的句子结构。还要涵盖秦始皇的主要成就，比如统一六国、统一文字、度量衡、修建长城、灵渠，以及他的统治手段，比如焚书坑儒，还有他的陵墓和长生不老的追求。

同时，要注意避免过于冗长，保持简要。可能需要分点列出主要成就，但用连贯的段落来表达。还要考虑用户可能的深层需求，比如对秦始皇的评价，是正面还是负面，或者想了解他的历史影响。因此，在介绍中可以适当提到他的功过，但保持中立，以秦始皇的视角来陈述。

另外，要确保历史准确性，比如统一的时间、主要政策和工程。需要确认这些信息是否正确，比如统一六国是在公元前221年，修建长城和灵渠的时间，以及焚书坑儒的背景。同时，提到他的陵墓和长生不老药的追求，这些也是重要的点。

最后，结尾部分要体现出秦始皇的威严和对后世的影响，比如“朕之功业，后世所铭记”，这样既符合角色设定，又满足用户对历史人物的了解需求。需要检查是否有遗漏的重要信息，并确保语言流畅自然，符合古代帝王的口吻。
</think>

*负手而立，目光如炬*

朕乃秦始皇，姓嬴名政，秦王之子。十三岁即位，二十二岁亲政，统一六国，废分封，立郡县，天下归一。朕命李斯整理文字，统一度量衡，筑长城以御匈奴，凿灵渠以通岭南，焚书坑儒以禁异端。朕欲长生不老，遣徐福东渡求仙药，虽未得，然朕之功业，后世所铭记。

*轻抚玉玺，神情威严*

朕之秦，乃万世一统之基。朕之名，后人或褒或贬，然朕之志，唯求华夏永固，万世一系。<|im_end|>


在基本掌握Unsloth的模型导入和对话方法后，接下来正式进入到Qwen3大模型高效微调流程中。

## Qwen3高效微调数据集创建

### 1. HuggingFace与ModelScope平台数据集介绍

&emsp;&emsp;若要选择公开数据集进行高效微调，则首先可以考虑HuggingFace和ModelScope平台上的数据集。其中HuggingFace上不仅保管了最大规模数量的数据集，而且HuggingFace的dataset工具，也是目前主流微调核心库如trl库默认支持的数据集格式。此外，ModelScope则是国内版的“HuggingFace”，拥有最大规模的中文数据集。

- HuggingFace数据集主页：https://huggingface.co/?activityType=all&feedType=following&trending=dataset

- 魔搭社区数据集主页：https://www.modelscope.cn/datasets

### 2. 数据集下载流程

Qwen3 具备推理模式和非推理模式。（非推理模式（普通聊天）、推理模式（深度思考））因此，应当使用两个数据集：

1. 使用 Open Math Reasoning 数据集，该数据集曾被用于赢得 AIMO（AI 数学奥林匹克 - 第二届进步奖）挑战！我们从中抽取了 10% 可验证的推理轨迹，这些轨迹是基于 DeepSeek R1 模型生成的，并且准确率超过 95%。数据集地址：https://huggingface.co/datasets/unsloth/OpenMathReasoning-mini

2. 利用 Maxime Labonne 的 FineTome-100k 数据集，该数据集风格类似 ShareGPT。但我们需要将其转换为 HuggingFace 通用的多轮对话格式。数据集地址：https://huggingface.co/datasets/mlabonne/FineTome-100k

在实际微调过程中，大多都会使用huggingface的datasets库进行数据集下载和管理，实际下载流程如下：

In [ ]:
!pip install --upgrade datasets huggingface_hub

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.3/512.3 kB 5.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.0/521.0 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 24.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.36.0
    Uninstalling huggingface-hub-0.36.0:
      Successfully uninstalled huggingface-hub-0.36.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.3.0
    Uninstalling datasets-4.3.0:
      Successfully uninstalled datasets-4.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.57.3 requires huggingface-hub<1.0,>=0.34.0, but you have huggingface-hub 1.2.3 which is incompatible.
unsloth-zoo 2026.1.1 requires datasets!=4.0.*,!

In [ ]:
# 设置 HTTP 和 HTTPS 代理
os.environ["HTTP_PROXY"] = "http://127.0.0.1:10080"
os.environ["HTTPS_PROXY"] = "http://127.0.0.1:10080"

In [ ]:
# 取消代理
os.environ.pop('HTTP_PROXY', None)
os.environ.pop('HTTPS_PROXY', None)
os.environ.pop('http_proxy', None) 
os.environ.pop('https_proxy', None)

- 或者去手动下载

&emsp;&emsp;`datasets` 是 Hugging Face 提供的一个高效数据处理库，专为机器学习和大语言模型（LLM）训练而设计。它支持加载、处理、转换和保存各种格式的数据（如 JSON、CSV、Parquet 等），并能与 `transformers` 模型无缝集成。通过 `datasets`，开发者可以快速完成数据清洗、切分、tokenization 等常见任务，大大提升训练效率，特别适合用于指令微调、对话生成、Function Calling 等任务的数据预处理。

In [ ]:
from datasets import load_dataset

然后分别下载并导入这两个库：

In [ ]:
!modelscope download --dataset AI-ModelScope/FineTome-100k

/root/miniconda3/lib/python3.12/site-packages/modelscope/utils/plugins.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources

 _   .-')                _ .-') _     ('-.             .-')                              _ (`-.    ('-.
( '.( OO )_             ( (  OO) )  _(  OO)           ( OO ).                           ( (OO  ) _(  OO)
 ,--.   ,--.).-'),-----. \     .'_ (,------.,--.     (_)---\_)   .-----.  .-'),-----.  _.`     \(,------.
 |   `.'   |( OO'  .-.  ',`'--..._) |  .---'|  |.-') /    _ |   '  .--./ ( OO'  .-.  '(__...--'' |  .---'
 |         |/   |  | |  ||  |  \  ' |  |    |  | OO )\  :` `.   |  |('-. /   |  | |  | |  /  | | |  |
 |  |'.'|  |\_) |  |\|  ||  |   ' |(|  '--. |  |`-' | '..`''.) /_) |OO  )\_) |  |\|  | |  |_.' |(|  '--.
 |  |   |  |  \ |  | |  ||  

In [ ]:
reasoning_dataset = load_dataset("/root/pandagpt_data/data/OpenMathReasoning-mini", split = "cot")

> 只下载包含cot的数据集

In [ ]:
non_reasoning_dataset = load_dataset("/root/.cache/modelscope/hub/datasets/AI-ModelScope/FineTome-100k", split = "train")

> 只下载train部分数据

然后输入数据集名称，即可查看数据集基本信息：

In [ ]:
reasoning_dataset

Dataset({
    features: ['expected_answer', 'problem_type', 'problem_source', 'generation_model', 'pass_rate_72b_tir', 'problem', 'generated_solution', 'inference_mode'],
    num_rows: 19252
})

加上索引则可以直接查看对应数据集信息：

In [ ]:
reasoning_dataset[0]

{'expected_answer': '14',
 'problem_type': 'has_answer_extracted',
 'problem_source': 'aops_c4_high_school_math',
 'generation_model': 'DeepSeek-R1',
 'pass_rate_72b_tir': '0.96875',
 'problem': 'Given $\\sqrt{x^2+165}-\\sqrt{x^2-52}=7$ and $x$ is positive, find all possible values of $x$.',
 'generated_solution': "<think>\nOkay, let's see. I need to solve the equation √(x² + 165) - √(x² - 52) = 7, and find all positive values of x. Hmm, radicals can be tricky, but maybe if I can eliminate the square roots by squaring both sides. Let me try that.\n\nFirst, let me write down the equation again to make sure I have it right:\n\n√(x² + 165) - √(x² - 52) = 7.\n\nOkay, so the idea is to isolate one of the radicals and then square both sides. Let me try moving the second radical to the other side:\n\n√(x² + 165) = 7 + √(x² - 52).\n\nNow, if I square both sides, maybe I can get rid of the square roots. Let's do that:\n\n(√(x² + 165))² = (7 + √(x² - 52))².\n\nSimplifying the left side:\n\nx² + 

能够看出这是一个基于DeepSeek R1回答的数学数据集，其中`problem`是问题，`generated_solution`是数学推导过程（即思考过程），而`expected_answer`则是最终的答案。该数据集总共接近2万条数据：

In [ ]:
len(reasoning_dataset)

19252

而对话数据集如下：

In [ ]:
non_reasoning_dataset

Dataset({
    features: ['conversations', 'source', 'score'],
    num_rows: 100000
})

In [ ]:
non_reasoning_dataset[0]

{'conversations': [{'from': 'human',
   'value': 'Explain what boolean operators are, what they do, and provide examples of how they can be used in programming. Additionally, describe the concept of operator precedence and provide examples of how it affects the evaluation of boolean expressions. Discuss the difference between short-circuit evaluation and normal evaluation in boolean expressions and demonstrate their usage in code. \n\nFurthermore, add the requirement that the code must be written in a language that does not support short-circuit evaluation natively, forcing the test taker to implement their own logic for short-circuit evaluation.\n\nFinally, delve into the concept of truthiness and falsiness in programming languages, explaining how it affects the evaluation of boolean expressions. Add the constraint that the test taker must write code that handles cases where truthiness and falsiness are implemented differently across different programming languages.'},
  {'from': 'gpt

其中每一条数据都是一个对话，包含一组或者多组ChatGPT的聊天信息，其中`from`代表是用户消息还是大模型回复消息，而`value`则是对应的文本。该对话数据集总共包含10万条数据：

In [ ]:
len(non_reasoning_dataset)

100000

能够看出dataset是一种类似json的数据格式，每条数据都以字段格式进行存储，在实际微调过程中，我们需要先将数据集的目标字段进行提取和拼接，然后加载到Qwen3模型的提示词模板中，并最终带入Unsloth进行微调。

### 2. 微调数据集清洗

&emsp;&emsp;接下来尝试对上述两个格式各异的数据集进行数据清洗，主要是围绕数据集进行数据格式的调整，便于后续带入Qwen3提示词模板。对于dataset格式的数据对象来说，可以先创建满足格式调整的函数，然后使用map方法对数据集格式进行调整。这里先创建generate_conversation函数，用于对reasoning_dataset中的每一条数据进行格式调整，即通过新创建一个新的特征conversations，来以对话形式保存历史问答数据：

In [ ]:
def generate_conversation(examples):
    problems  = examples["problem"]
    solutions = examples["generated_solution"]
    conversations = []
    for problem, solution in zip(problems, solutions):
        conversations.append([
            {"role" : "user",      "content" : problem},
            {"role" : "assistant", "content" : solution},
        ])
    return { "conversations": conversations, }

In [ ]:
reasoning_dataset[0]

{'expected_answer': '14',
 'problem_type': 'has_answer_extracted',
 'problem_source': 'aops_c4_high_school_math',
 'generation_model': 'DeepSeek-R1',
 'pass_rate_72b_tir': '0.96875',
 'problem': 'Given $\\sqrt{x^2+165}-\\sqrt{x^2-52}=7$ and $x$ is positive, find all possible values of $x$.',
 'generated_solution': "<think>\nOkay, let's see. I need to solve the equation √(x² + 165) - √(x² - 52) = 7, and find all positive values of x. Hmm, radicals can be tricky, but maybe if I can eliminate the square roots by squaring both sides. Let me try that.\n\nFirst, let me write down the equation again to make sure I have it right:\n\n√(x² + 165) - √(x² - 52) = 7.\n\nOkay, so the idea is to isolate one of the radicals and then square both sides. Let me try moving the second radical to the other side:\n\n√(x² + 165) = 7 + √(x² - 52).\n\nNow, if I square both sides, maybe I can get rid of the square roots. Let's do that:\n\n(√(x² + 165))² = (7 + √(x² - 52))².\n\nSimplifying the left side:\n\nx² + 

In [ ]:
reasoning_data = reasoning_dataset.map(generate_conversation, batched = True)

此时历史问答数据如下：

In [ ]:
reasoning_data["conversations"]

Column([[{'content': 'Given $\\sqrt{x^2+165}-\\sqrt{x^2-52}=7$ and $x$ is positive, find all possible values of $x$.', 'role': 'user'}, {'content': "<think>\nOkay, let's see. I need to solve the equation √(x² + 165) - √(x² - 52) = 7, and find all positive values of x. Hmm, radicals can be tricky, but maybe if I can eliminate the square roots by squaring both sides. Let me try that.\n\nFirst, let me write down the equation again to make sure I have it right:\n\n√(x² + 165) - √(x² - 52) = 7.\n\nOkay, so the idea is to isolate one of the radicals and then square both sides. Let me try moving the second radical to the other side:\n\n√(x² + 165) = 7 + √(x² - 52).\n\nNow, if I square both sides, maybe I can get rid of the square roots. Let's do that:\n\n(√(x² + 165))² = (7 + √(x² - 52))².\n\nSimplifying the left side:\n\nx² + 165 = 49 + 14√(x² - 52) + (√(x² - 52))².\n\nThe right side is expanded using the formula (a + b)² = a² + 2ab + b². So the right side becomes 7² + 2*7*√(x² - 52) + (√(x²

In [ ]:
reasoning_data["conversations"][0]

[{'content': 'Given $\\sqrt{x^2+165}-\\sqrt{x^2-52}=7$ and $x$ is positive, find all possible values of $x$.',
  'role': 'user'},
 {'content': "<think>\nOkay, let's see. I need to solve the equation √(x² + 165) - √(x² - 52) = 7, and find all positive values of x. Hmm, radicals can be tricky, but maybe if I can eliminate the square roots by squaring both sides. Let me try that.\n\nFirst, let me write down the equation again to make sure I have it right:\n\n√(x² + 165) - √(x² - 52) = 7.\n\nOkay, so the idea is to isolate one of the radicals and then square both sides. Let me try moving the second radical to the other side:\n\n√(x² + 165) = 7 + √(x² - 52).\n\nNow, if I square both sides, maybe I can get rid of the square roots. Let's do that:\n\n(√(x² + 165))² = (7 + √(x² - 52))².\n\nSimplifying the left side:\n\nx² + 165 = 49 + 14√(x² - 52) + (√(x² - 52))².\n\nThe right side is expanded using the formula (a + b)² = a² + 2ab + b². So the right side becomes 7² + 2*7*√(x² - 52) + (√(x² - 52

接下来将其带入Qwen3的提示词模板中进行转化：

In [ ]:
# 查看数据结构
print(type(reasoning_data["conversations"]))
print(len(reasoning_data["conversations"]))
print(reasoning_data["conversations"][:2])  # 查看前两条

<class 'datasets.arrow_dataset.Column'>
19252
[[{'content': 'Given $\\sqrt{x^2+165}-\\sqrt{x^2-52}=7$ and $x$ is positive, find all possible values of $x$.', 'role': 'user'}, {'content': "<think>\nOkay, let's see. I need to solve the equation √(x² + 165) - √(x² - 52) = 7, and find all positive values of x. Hmm, radicals can be tricky, but maybe if I can eliminate the square roots by squaring both sides. Let me try that.\n\nFirst, let me write down the equation again to make sure I have it right:\n\n√(x² + 165) - √(x² - 52) = 7.\n\nOkay, so the idea is to isolate one of the radicals and then square both sides. Let me try moving the second radical to the other side:\n\n√(x² + 165) = 7 + √(x² - 52).\n\nNow, if I square both sides, maybe I can get rid of the square roots. Let's do that:\n\n(√(x² + 165))² = (7 + √(x² - 52))².\n\nSimplifying the left side:\n\nx² + 165 = 49 + 14√(x² - 52) + (√(x² - 52))².\n\nThe right side is expanded using the formula (a + b)² = a² + 2ab + b². So the right s

In [ ]:
all_conversations = list(reasoning_data["conversations"])  # 转换为Python列表

# 批量处理
rendered_all = tokenizer.apply_chat_template(
    all_conversations,  # 列表的列表
    tokenize=False,
)

In [ ]:
reasoning_conversations = rendered_all  

In [ ]:
reasoning_conversations = tokenizer.apply_chat_template(
    reasoning_data["conversations"],
    tokenize = False,
)

UndefinedError: 'list object' has no attribute 'content'

最后创建的数据就是一个包含多个对话信息的list：

In [ ]:
reasoning_conversations[0]

"<|im_start|>user\nGiven $\\sqrt{x^2+165}-\\sqrt{x^2-52}=7$ and $x$ is positive, find all possible values of $x$.<|im_end|>\n<|im_start|>assistant\n<think>\nOkay, let's see. I need to solve the equation √(x² + 165) - √(x² - 52) = 7, and find all positive values of x. Hmm, radicals can be tricky, but maybe if I can eliminate the square roots by squaring both sides. Let me try that.\n\nFirst, let me write down the equation again to make sure I have it right:\n\n√(x² + 165) - √(x² - 52) = 7.\n\nOkay, so the idea is to isolate one of the radicals and then square both sides. Let me try moving the second radical to the other side:\n\n√(x² + 165) = 7 + √(x² - 52).\n\nNow, if I square both sides, maybe I can get rid of the square roots. Let's do that:\n\n(√(x² + 165))² = (7 + √(x² - 52))².\n\nSimplifying the left side:\n\nx² + 165 = 49 + 14√(x² - 52) + (√(x² - 52))².\n\nThe right side is expanded using the formula (a + b)² = a² + 2ab + b². So the right side becomes 7² + 2*7*√(x² - 52) + (√(x² 

In [ ]:
len(reasoning_conversations)

19252

之后即可带入这些数据进行微调。能看出每条数据的格式都和Unsloth底层对话API创建的数据格式类似，之后我们或许可以借助Unsloth底层对话API来创建微调数据集。

&emsp;&emsp;然后继续处理non_reasoning_conversations数据集，由于该数据集采用了sharegpt对话格式，因此可以直接借助Unsloth的standardize_sharegpt库进行数据集的格式转化，转化效果如下所示：

In [ ]:
from unsloth.chat_templates import standardize_sharegpt

In [ ]:
dataset = standardize_sharegpt(non_reasoning_dataset)

Unsloth: Standardizing formats (num_proc=256):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [ ]:
dataset["conversations"][0]

[{'content': 'Explain what boolean operators are, what they do, and provide examples of how they can be used in programming. Additionally, describe the concept of operator precedence and provide examples of how it affects the evaluation of boolean expressions. Discuss the difference between short-circuit evaluation and normal evaluation in boolean expressions and demonstrate their usage in code. \n\nFurthermore, add the requirement that the code must be written in a language that does not support short-circuit evaluation natively, forcing the test taker to implement their own logic for short-circuit evaluation.\n\nFinally, delve into the concept of truthiness and falsiness in programming languages, explaining how it affects the evaluation of boolean expressions. Add the constraint that the test taker must write code that handles cases where truthiness and falsiness are implemented differently across different programming languages.',
  'role': 'user'},
 {'content': 'Boolean operators a

接下来即可直接带入Qwen3对话模板中进行格式调整：

In [ ]:
from unsloth.chat_templates import standardize_sharegpt

# 标准化数据集
dataset = standardize_sharegpt(non_reasoning_dataset)

# 将Column对象转换为Python列表
conversations_list = list(dataset["conversations"])

# 应用chat template
non_reasoning_conversations = tokenizer.apply_chat_template(
    conversations_list,
    tokenize=False,
)

print(f"成功处理 {len(non_reasoning_conversations)} 个对话")

成功处理 100000 个对话


In [ ]:
non_reasoning_conversations = tokenizer.apply_chat_template(
    dataset["conversations"],
    tokenize = False,
)

UndefinedError: 'list object' has no attribute 'content'

最终每一条数据格式如下：

In [ ]:
non_reasoning_conversations[0]

'<|im_start|>user\nExplain what boolean operators are, what they do, and provide examples of how they can be used in programming. Additionally, describe the concept of operator precedence and provide examples of how it affects the evaluation of boolean expressions. Discuss the difference between short-circuit evaluation and normal evaluation in boolean expressions and demonstrate their usage in code. \n\nFurthermore, add the requirement that the code must be written in a language that does not support short-circuit evaluation natively, forcing the test taker to implement their own logic for short-circuit evaluation.\n\nFinally, delve into the concept of truthiness and falsiness in programming languages, explaining how it affects the evaluation of boolean expressions. Add the constraint that the test taker must write code that handles cases where truthiness and falsiness are implemented differently across different programming languages.<|im_end|>\n<|im_start|>assistant\n<think>\n\n</th

In [ ]:
print(len(reasoning_conversations))
print(len(non_reasoning_conversations))

19252
100000


自此即完成了每个数据集的格式调整工作，不过这两个数据集并不均衡，能看得出非推理类数据集的长度更长。我们假设希望模型保留一定的推理能力，但又特别希望它作为一个聊天模型来使用。因此，我们需要定义一个 仅聊天数据的比例。目标是从两个数据集中构建一个混合训练集。这里我们可以设定一个 25% 推理数据、75% 聊天数据的比例：也就是说，从推理数据集中抽取 25%（或者说，抽取占比为 100% - 聊天数据占比 的部分），最后将这两个数据集合并起来即可。这里我们需要先将上述list格式的数据转化为pd.Series数据，然后进行采样，并最终将其转化为dataset类型对象。（此外也可以先转化为dataset对象类型，然后再进行采样）

In [ ]:
chat_percentage = 0.75

In [ ]:
import pandas as pd
non_reasoning_subset = pd.Series(non_reasoning_conversations) 
# pd.Series:
# 普通的 Python 列表 (list)：就像一排没有编号的储物柜，你只能通过位置（0, 1, 2...）来找东西。
# Pandas Series：就像一列有标签的 Excel 表格。它不仅存数据，还给每个数据配了一个“索引”（Index）。
# 为了使用 Pandas 提供的高级操作功能，这些功能是原生 Python 列表不具备的：
# .sample()：快速进行随机采样。在你的代码里，你用它从大量的普通对话中精确抽取出一部分，用来匹配推理数据的比例。
# .concat()：像接火车一样把两个 Series（推理数据 + 抽取的普通数据）拼在一起。
# .name = "text"：直接给这一列数据起个名字。这非常重要，因为后续转化为 Hugging Face Dataset 时，它会自动识别 "text" 作为列名，这也是模型训练器（Trainer）默认寻找的目标。
non_reasoning_subset = non_reasoning_subset.sample(
    int(len(reasoning_conversations) * (1.0 - chat_percentage)),
    random_state = 2407,
)

然后进行拼接和转化：

In [ ]:
data = pd.concat([
    pd.Series(reasoning_conversations),
    pd.Series(non_reasoning_subset)
])
data.name = "text"

from datasets import Dataset
combined_dataset = Dataset.from_pandas(pd.DataFrame(data))
combined_dataset = combined_dataset.shuffle(seed = 3407)

转化后数据集如下所示：

In [ ]:
len(combined_dataset)

24065

In [ ]:
type(combined_dataset)

datasets.arrow_dataset.Dataset

In [ ]:
combined_dataset

Dataset({
    features: ['text', '__index_level_0__'],
    num_rows: 24065
})

对话类数据集

In [ ]:
combined_dataset[0]

{'text': "<|im_start|>user\nCalculate the pH during a titration when 9.54 mL of a 0.15 M HCl solution has reacted with 22.88 mL of a 0.14 M NaOH solution?<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\nStart by calculating the number of moles of hydrochloric acid and the number of moles of sodium hydroxide that took part in the reaction.\n\n9.54 mL of 0.15 M HCl solution contains:\n\n0.15 mol HCl / 1000 mL HCl solution * 9.54 mL HCl solution = 0.001431 mol HCl\n\n22.88 mL of 0.14 M NaOH solution contains:\n\n0.14 mol NaOH / 1000 mL NaOH solution * 22.88 mL NaOH solution = 0.003203 mol NaOH\n\nHydrochloric acid and sodium hydroxide neutralize each other in a 1:1 mole ratio. This means that in order to have a complete neutralization, you need to mix equal numbers of moles of hydrochloric acid and of sodium hydroxide. In this case, you have more moles of sodium hydroxide than of hydrochloric acid, so the acid will act as a limiting reagent, i.e. it will be completely consumed be

推理类数据集

In [ ]:
combined_dataset[2]

{'text': "<|im_start|>user\nDetermine the surface area of the portion of the plane $2x + 3y + 6z = 9$ that lies in the first octant.<|im_end|>\n<|im_start|>assistant\n<think>\nOkay, let's see. I need to find the surface area of the part of the plane 2x + 3y + 6z = 9 that's in the first octant. Hmm, first octant means all x, y, z are non-negative, right? So I should probably figure out the region on the plane where x ≥ 0, y ≥ 0, z ≥ 0. Then find the area of that triangular-like surface.\n\nWait, planes in 3D can form triangles when intersected with the first octant, depending on where they cut the axes. Maybe I should find the intercepts first. Let me recall that to find intercepts, set the other variables to zero. \n\nFor the x-intercept, set y=0 and z=0. Then 2x = 9, so x = 9/2 = 4.5. \n\nFor the y-intercept, set x=0 and z=0. Then 3y = 9, so y = 3.\n\nFor the z-intercept, set x=0 and y=0. Then 6z = 9, so z = 1.5.\n\nSo the plane intersects the x-axis at (4.5, 0, 0), y-axis at (0, 3, 0

其中text字段就是后续带入微调的字段。

- 数据集保存

&emsp;&emsp;最后即可将清洗好的数据集进行本地保存：

In [ ]:
combined_dataset.save_to_disk("cleaned_qwen3_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/24065 [00:00<?, ? examples/s]

后续使用时即可使用如下代码进行读取：

In [ ]:
from datasets import load_from_disk
combined_dataset = load_from_disk("cleaned_qwen3_dataset")

In [ ]:
type(combined_dataset)

datasets.arrow_dataset.Dataset

## 六、Qwen3推理能力高效微调流程

&emsp;&emsp;准备完数据之后，即可开始进行微调。这里我们先进行少量数据微调测试，程序能够基本跑通后，我们再进行大规模数据集微调。

### 1. Unsloth微调流程实践

#### Step 1. 进行LoRA参数注入

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,               # 秩 (Rank)：选择大于 0 的整数。建议值为 8, 16, 32, 64, 128
    target_modules = [    # 目标模块：LoRA 权重将应用到模型中的哪些层
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = 32,      # 缩放系数：最佳实践是选择 alpha = r 或者 r * 2
    lora_dropout = 0,     # Dropout 丢弃率：支持任何数值，但设为 0 是经过 Unsloth 特别优化的
    bias = "none",        # 偏置项：支持任何选项，但设为 "none" 是经过优化的
    
    # [新特性] "unsloth" 模式可减少 30% 显存占用，并支持 2 倍大的 Batch Size！
    use_gradient_checkpointing = "unsloth", # 面对长文本时，开启 True 或 "unsloth" 可节省显存
    
    random_state = 3407,  # 随机种子：固定随机性以便复现实验结果
    use_rslora = False,   # 是否使用秩稳定 LoRA (Rank Stabilized LoRA)
    loftq_config = None,  # 是否使用 LoftQ（量化初始权重的一种方法）
)

Unsloth 2026.1.1 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


#### Step 2. 设置微调参数

In [ ]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = combined_dataset,
    eval_dataset = None,      # 如果有验证集，可以在这里设置
    args = SFTConfig(
        dataset_text_field = "text",       # 指定数据集中包含训练文本的字段名
        per_device_train_batch_size = 2,  # 每个设备（GPU）的训练批次大小
        gradient_accumulation_steps = 4,  # 梯度累积步数：累积 4 步更新一次参数，模拟更大的 Batch Size
        warmup_steps = 5,                 # 预热步数：前 5 步学习率缓慢上升，防止训练初期不稳定
        
        # num_train_epochs = 1,           # 完整训练轮数：训练完整个数据集一遍
        max_steps = 5,                    # 最大训练步数：仅用于快速测试，正式训练请注释掉此行并开启 epoch
        
        learning_rate = 2e-4,             # 学习率：LoRA 常用 2e-4；长文本或大规模训练建议降至 2e-5
        logging_steps = 1,                # 日志记录频率：每步都打印训练状态（如 Loss 值）
        optim = "adamw_8bit",             # 优化器：使用 8-bit AdamW，显著节省显存空间
        weight_decay = 0.01,              # 权重衰减：防止过拟合的正则化手段
        lr_scheduler_type = "linear",     # 学习率调度策略：线性下降
        seed = 3407,                      # 随机种子：固定实验结果，方便复现
        report_to = "wandb",              # 实验监控：同步到 Weights & Biases 平台，不需要可设为 "none"
    ),
)

其中`SFTTrainer`：一个专门为指令微调设计的训练器，封装了 Hugging Face 的 `Trainer`，而`SFTConfig`：配置训练参数的专用类，功能类似 `TrainingArguments`。而SFTConfig核心参数解释如下：

| 参数名                             | 含义                                              |
| ------------------------------- | ----------------------------------------------- |
| `dataset_text_field="text"`     | 数据集中用于训练的字段名称，如 `text` 或 `prompt`               |
| `per_device_train_batch_size=2` | 每张 GPU 上的 batch size 是 2                        |
| `gradient_accumulation_steps=4` | 梯度累计 4 次后才进行一次反向传播（等效于总 batch size = 2 × 4 = 8） |
| `warmup_steps=5`                | 前 5 步进行 warmup（缓慢提升学习率）                         |
| `max_steps=30`                  | 最多训练 30 步（适合调试或快速实验）                            |
| `learning_rate=2e-4`            | 初始学习率（短训练可用较高值）                                 |
| `logging_steps=1`               | 每训练 1 步就打印一次日志                                  |
| `optim="adamw_8bit"`            | 使用 8-bit AdamW 优化器（节省内存，Unsloth 支持）             |
| `weight_decay=0.01`             | 权重衰减，用于防止过拟合                                    |
| `lr_scheduler_type="linear"`    | 线性学习率调度器（从高到低线性下降）                              |
| `seed=3407`                     | 固定随机种子，确保结果可复现                                  |
| `report_to="none"`              | 不使用 WandB 或 TensorBoard 等日志平台（可改为 `"wandb"`）    |

此时基本训练过程为：
1. 从 `combined_dataset` 中取出一批样本（2 条）
2. 重复上面过程 4 次（`gradient_accumulation_steps=4`）
3. 将累计的梯度用于更新模型一次参数（等效于一次大 batch 更新）
4. 重复上述过程，直到 `max_steps=30` 停止

此时显存占用如下：

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4090 D. Max memory = 23.516 GB.
19.152 GB of memory reserved.


#### Step 3. 【可选】设置wandb

&emsp;&emsp;接下来可继续设置wandb用于进行模型训练过程关键信息记录。

In [ ]:
import wandb

In [ ]:
os.environ["WANDB_NOTEBOOK_NAME"] = "Qwen3_unsloth.ipynb"

In [ ]:
wandb.login()  # 会自动使用环境变量中的key

True

In [ ]:
wandb.login(key="4b62572b8426ff59ce46fec93a00eb1feecc026a")

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find Qwen3高效微调（下）.ipynb.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 2323365771 (2323365771-ff) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
wandb.login(key="c27c8c652e21d1e11e4e3889ac4ce9d0d1f01e7d")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 2737609908 (2737609908-northwestern-polytechnical-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
run = wandb.init(project='Fine-tune-Qwen-8B-4bit on Combined Dataset', )

#### Step 4. 微调执行流程

&emsp;&emsp;一切准备就绪后，接下来即可开始进行微调。由于本次微调总共只运行30个step，整个过程并不会很长，实际执行过程如下：

In [ ]:
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 24,065 | Num Epochs = 1 | Total steps = 5
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 87,293,952 of 8,278,029,312 (1.05% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,0.341300
2,0.447100
3,0.407200
4,0.429500
5,0.383200


train/epoch,▁▃▅▆██
train/global_step,▁▃▅▆██
train/grad_norm,▄██▁▆
train/learning_rate,▁▃▅▆█
train/loss,▁█▅▇▄
total_flos,8404579976785920.0
train/epoch,0.00166
train/global_step,5
train/grad_norm,0.04633
train/learning_rate,0.00016
train/loss,0.3832


- 微调期间显存占用检测

In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

439.9436 seconds used for training.
7.33 minutes used for training.
Peak reserved memory = 19.152 GB.
Peak reserved memory for training = 11.045 GB.
Peak reserved memory % of max memory = 81.442 %.
Peak reserved memory for training % of max memory = 46.968 %.


- 查看wandb记录结果

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250507222022165.png" alt="image-20250507222022165" style="zoom:50%;" />

能够看到loss波动下降，整个训练过程属于正常情况。

#### Step 5. 模型对话测试

In [ ]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen3ForCausalLM(
      (model): Qwen3Model(
        (embed_tokens): Embedding(151936, 4096, padding_idx=151654)
        (layers): ModuleList(
          (0-2): 3 x Qwen3DecoderLayer(
            (self_attn): Qwen3Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.

而当我们完成训练后，即可尝试进行模型对话：

In [ ]:
messages = [
    {"role" : "user", "content" : "Solve (x + 2)^3 = 0."}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    enable_thinking = False, # Disable thinking
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 256, # Increase for longer outputs!
    temperature = 0.7, top_p = 0.8, top_k = 20, # For non thinking
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

To solve the equation \((x + 2)^3 = 0\), we start by recognizing that the left-hand side is a cube of a binomial. A cube of a binomial is zero if and only if the binomial itself is zero. Therefore, we set the binomial equal to zero:

\[
x + 2 = 0
\]

Next, we solve for \(x\) by isolating it on one side of the equation. We do this by subtracting 2 from both sides:

\[
x = -2
\]

Thus, the solution to the equation \((x + 2)^3 = 0\) is:

\[
\boxed{-2}
\]<|im_end|>


In [ ]:
messages = [
    {"role" : "user", "content" : "Solve (x + 2)^2 = 0."}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    enable_thinking = True, # Disable thinking
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 20488, # Increase for longer outputs!
    temperature = 0.6, top_p = 0.95, top_k = 20, # For thinking
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

<think>
Okay, let's see. I need to solve the equation (x + 2)^2 = 0. Hmm, how do I approach this? Well, when you have something squared equals zero, that means the inside of the square has to be zero because any real number squared is non-negative, and the only way for it to be zero is if the number itself is zero. So, if (x + 2)^2 = 0, then x + 2 must equal zero. That makes sense.

So, let's write that out. If (x + 2)^2 = 0, then taking the square root of both sides should give me x + 2 = 0. Wait, but when you take the square root of both sides, you have to consider both the positive and negative roots, right? But in this case, the square of a real number is zero only if the number itself is zero. So maybe there's only one solution here. Let me check that.

Alternatively, maybe I can expand the left side to see if that helps. Let's try expanding (x + 2)^2. That would be x^2 + 4x + 4. So the equation becomes x^2 + 4x + 4 = 0. Now, if I factor this quadratic equation, I can see if it fa

In [ ]:
messages = [
    {"role" : "user", "content" : "Determine the surface area of the portion of the plane $2x + 3y + 6z = 9$ that lies in the first octant."}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    enable_thinking = True, # Disable thinking
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 20488, # Increase for longer outputs!
    temperature = 0.6, top_p = 0.95, top_k = 20, # For thinking
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

<think>
Okay, so I need to find the surface area of the portion of the plane 2x + 3y + 6z = 9 that's in the first octant. Hmm, first octant means x, y, z are all non-negative, right? So the plane intersects the x, y, and z axes at certain points, and I need to figure out the area of the triangle formed by those intercepts. Wait, is that right? Because in the first octant, the plane should form a triangular region, and the surface area would be the area of that triangle. But I need to confirm that.

First, let me find the intercepts. For the x-intercept, set y = 0 and z = 0. Then 2x = 9, so x = 9/2. Similarly, y-intercept: set x = 0 and z = 0, so 3y = 9, y = 3. Z-intercept: set x = 0 and y = 0, so 6z = 9, z = 3/2. So the plane intersects the axes at (9/2, 0, 0), (0, 3, 0), and (0, 0, 3/2). These three points form a triangle in the first octant. So the surface area is the area of this triangle. But wait, is that the case? Because the surface area of a plane in the first octant would actu

KeyboardInterrupt: 

#### Step 6. 模型大规模微调

&emsp;&emsp;接下来继续深入进行训练，此处考虑训练完一整个epoch，总共约8小时左右，训练流程如下所示：

In [ ]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = combined_dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 2, # Use GA to mimic batch size!
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=176): 100%|██████████| 24065/24065 [07:06<00:00, 56.44 examples/s]


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 24,065 | Num Epochs = 1 | Total steps = 3,008
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 2 x 1) = 8
 "-____-"     Trainable parameters = 268,435,456/32,000,000,000 (0.84% trained)


Step,Training Loss
1,0.378600
2,0.439700
3,0.448800
4,0.425000
5,0.373800
6,0.355300
7,0.424800
8,0.382300
9,0.350200
10,0.420600


此时训练完成后再进行对话，能明显看出模型当前数学性能有所提升，具体问答效果如下：

In [ ]:
messages = [
    {"role" : "user", "content" : "Determine the surface area of the portion of the plane $2x + 3y + 6z = 9$ that lies in the first octant."}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    enable_thinking = True, # Disable thinking
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 20488, # Increase for longer outputs!
    temperature = 0.6, top_p = 0.95, top_k = 20, # For thinking
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

<think>
Okay, so I need to find the surface area of the portion of the plane 2x + 3y + 6z = 9 that lies in the first octant. Hmm, first octant means x, y, z are all non-negative, right? So the plane intersects the coordinate axes at some points, and the region we're interested in is the triangle formed by those intercepts. But wait, surface area of a plane... isn't that just the area of that triangular region? But maybe I need to confirm if it's a triangle. Let me think.

First, let's find the intercepts. For the x-intercept, set y=0 and z=0. So 2x = 9 → x = 9/2. Similarly, y-intercept: set x=0, z=0. 3y = 9 → y = 3. Z-intercept: set x=0, y=0. 6z = 9 → z = 9/6 = 3/2. So the intercepts are at (9/2, 0, 0), (0, 3, 0), and (0, 0, 3/2). So the portion of the plane in the first octant is a triangle connecting these three points. Therefore, the surface area should be the area of this triangle. But wait, the problem says "surface area of the portion of the plane," which is a flat surface, so ye

#### Step 7.模型保存

微调结束后即可进行模型保存，由于我们训练的LoRA本身是FP16精度，因此模型需要保存为fp16精度格式，才能完整保留模型当前性能：

In [ ]:
model.save_pretrained_merged(save_directory = "/root/pandagpt_data/qwen/Qwen3-8B-finetuned-fp16", 
                             tokenizer = tokenizer, 
                             save_method = "merged_16bit")

In [ ]:
# 基本保存
save_path = "/root/pandagpt_data/qwen/Qwen3-8B-finetuned"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print(f"模型保存到: {save_path}")

模型保存到: /root/pandagpt_data/qwen/Qwen3-8B-finetuned


> - 正在将原始 4bit 权重与微调产生的 LoRA adapter 合并，并转换为 16bit（FP16）精度，用于部署
> - 会尽量使用不超过 703.3 MB（最大上限是你设备的 70～75% 内存）来进行保存操作，避免崩溃或卡死
> - 开始保存模型，会进行合并+格式转换，时间取决于模型大小（比如 Qwen3-32B 预计要几分钟）
> - 如果模型太大而内存不足，Unsloth 会自动启用“磁盘保存模式”，避免占用过多内存
> - tokenizer 保存完毕

In [ ]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ['UNSLOTH_USE_MODELSCOPE'] = '1'
print("1.1")
from unsloth import FastLanguageModel
import torch
print("1.1")
max_seq_length = 8192
dtype = None
load_in_4bit = True
model, tokenizer = FastLanguageModel.from_pretrained(
    # model_name = "./Qwen3-0.6B-unsloth-bnb-4bit",
    model_name = "/root/pandagpt_data/qwen/Qwen3-8B-finetuned",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    local_files_only = True,  # 添加这一行
)

1.1
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


Exception: huggingface-hub>=0.34.0,<1.0 is required for a normal functioning of this module, but found huggingface-hub==1.2.3.
Try: `pip install transformers -U` or `pip install -e '.[dev]'` if you're working with git main

#### Step 8.模型性能测试

接下来即可使用vllm对其进行调用，并借助EvalScope进行测试。需要借助vLLM调用导出后的模型，然后借助evalscope进行测试。

```python
from evalscope import TaskConfig, run_task
task_cfg = TaskConfig(
    model='./Qwen3-32B-finetuned-fp16',
    api_url='http://127.0.0.1:8000/v1/chat/completions',
    eval_type='service',
    datasets=[
        'data_collection',
    ],
    dataset_args={
        'data_collection': {
            'dataset_id': 'modelscope/EvalScope-Qwen3-Test',
            'filters': {'remove_until': '</think>'}  # 过滤掉思考的内容
        }
    },
    eval_batch_size=128,
    generation_config={
        'max_tokens': 30000,  # 最大生成token数，建议设置为较大值避免输出截断
        'temperature': 0.6,  # 采样温度 (qwen 报告推荐值)
        'top_p': 0.95,  # top-p采样 (qwen 报告推荐值)
        'top_k': 20,  # top-k采样 (qwen 报告推荐值)
        'n': 1,  # 每个请求产生的回复数量
    },
    timeout=60000,  # 超时时间
    stream=True,  # 是否使用流式输出
    limit=2000,  # 设置为2000条数据进行测试
)

run_task(task_cfg=task_cfg)
```

测试后即可在前端中观察测评报告：
```bash
evalscope app
```

Qwen3中文法律高效微调实战

需要重启Jupyter Kernel。

### 1. 案例介绍与数据集介绍

- 数据集样例：

<center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250429221646854.png" alt="image-20250429221646854" style="zoom:50%;" />

> {"input": "**法律问题**  \n2016年4月20日下午，吸毒人员陈某某1电话联系被告人李某购买甲基苯丙胺，李某随后与陈某某2共同驾车至宁远县禾亭镇陈家村，由李某直接向陈某某1交付4.3克甲基苯丙胺并收取500元毒资，后又将剩余毒品转交至第三方“李2某”处。公安机关根据举报抓获李某，并从陈某某1处查获含袋重4.3克的甲基苯丙胺。根据《刑法》第347条，李某的行为是否构成走私、贩卖、运输、制造毒品罪？其被判处6个月有期徒刑及3000元罚金的量刑是否适当？", "output": "<think>**  \n1. **犯罪构成要件分析**  \n   - **客观要件**：  \n     - **行为**：李某直接参与毒品交易（接收订单、送货、收款），并协助转移剩余毒品，符合“贩卖”和“运输”行为特征。  \n     - **结果**：实际交付4.3克甲基苯丙胺，毒品已流入社会。  \n     - **因果关系**：李某的行为直接促成毒品交易完成。  \n   - **主观要件**：李某明知甲基苯丙胺为毒品仍主动参与交易，具有贩卖毒品的直接故意。  \n\n2. **相关法条适用分析**  \n   - 根据《刑法》第347条，贩卖甲基苯丙胺无论数量多少均应追究刑事责任。本案毒品数量为4.3克（含包装），属“少量毒品”，但法条未设定入罪数量下限。  \n\n3. **量刑情节分析**  \n   - **从轻情节**：  \n     - 涉案毒品数量较少（4.3克）；  \n     - 无证据表明李某系主犯或组织者（行为表现为协助实施）。  \n   - **从重情节**：  \n     - 毒品已实际交付，社会危害性成立；  \n     - 存在毒品流转的连续性（向“李2某”转移剩余毒品）。  \n\n4. **判决适当性评价**  \n   - 量刑符合《刑法》第347条及司法解释对“少量毒品”的标准，6个月有期徒刑处于“三年以下有期徒刑、拘役或管制”的法定刑幅度内；  \n   - 罚金3000元与毒品价值（500元交易额）比例适当，体现罪责刑相适应原则。  \n\n**</think>\n\n---\n\n**\n**  \n\n李某的行为构成贩卖、运输毒品罪，罪名认定正确。判决量刑适当，理由如下：  \n1. 其行为完全符合贩卖、运输毒品罪的构成要件；  \n2. 6个月有期徒刑及3000元罚金与涉案毒品数量、犯罪情节及社会危害性相匹配，符合《刑法》第347条对“少量毒品”的处罚标准。"}

> {"input": "**问题：**  \n被告人龚某某于2012年11月7日在常德市武陵区战备桥“常德市邮政东区投递部”前，以购买被害人张某甲出售的白色神龙富康牌小车（车牌湘JB2898）为名，谎称试车后趁机将车辆骗走。经鉴定，车辆价值人民币5100元。案发后，龚某某如实供述犯罪事实，赔偿被害人4800元并取得谅解。根据《刑法》第266条，龚某某的行为是否构成诈骗罪？其被判处2个月有期徒刑并处罚金2000元的量刑是否适当？", "output": "<think>**  \n1. **犯罪构成要件分析**  \n   - **客观要件**：  \n     - **行为**：龚某某虚构“买车”意图，以试车为名非法占有车辆（欺骗手段+转移财产控制）。  \n     - **结果**：被害人实际丧失车辆所有权（价值5100元）。  \n     - **因果关系**：欺骗行为直接导致被害人基于错误认识交付车辆。  \n   - **主观要件**：龚某某具有非法占有的直接故意（明知无购买意愿仍骗取车辆）。  \n\n2. **相关法条适用分析**  \n   - 《刑法》第266条诈骗罪要求“以非法占有为目的，用虚构事实或隐瞒真相的方法骗取数额较大的公私财物”。本案中：  \n     - 欺骗手段（假意买车+试车）符合“虚构事实”。  \n     - 涉案金额5100元，超过诈骗罪“数额较大”标准（司法解释中通常为3000元以上）。  \n\n3. **量刑情节分析**  \n   - **从轻情节**：  \n     - 如实供述（可认定为坦白，依法从轻）。  \n     - 赔偿损失并取得谅解（酌定从轻）。  \n   - **无从重情节**：无证据显示暴力、多次犯罪等加重情节。  \n\n4. **判决适当性评价**  \n   - 基准刑参考：诈骗5100元，法定刑为“3年以下有期徒刑、拘役或管制，并处或单处罚金”。  \n   - 从轻幅度：坦白+赔偿谅解可减少基准刑30%以下。  \n   - 实际量刑：2个月有期徒刑（接近拘役下限）+罚金2000元，符合从轻后的合理范围，且罚金与犯罪金额比例适当。  \n\n**</think>\n\n**\n**  \n\n龚某某的行为构成诈骗罪，其以虚假购车名义骗取他人财物，涉案金额达到“数额较大”标准，符合《刑法》第266条的犯罪构成。法院判处2个月有期徒刑并处罚金2000元适当，理由包括：犯罪金额较低、坦白情节、全额赔偿及取得谅解等从轻因素，量刑在法律框架内且体现宽严相济原则。"}

数据主要包含两个部分：input、output，其中output包含<think>思考过程</think>与最终结果输出。

接下来我们尝试围绕该数据集进行模型法律能力微调，并由此实操大模型问答风格与知识灌注能力微调。

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

> 由于当前实验环境是多卡环境，而动态量化模型只支持单卡运行，因此这里先设置接下来运行的GPU编号。

### Step 1.模型导入

&emsp;&emsp;首先进行模型导入：

In [ ]:
from unsloth import FastLanguageModel
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/root/miniconda3/envs/unsloth/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
max_seq_length = 8192
dtype = None
load_in_4bit = True

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "./Qwen3-8B-unsloth-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.4.7: Fast Qwen3 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA H800 PCIe. Num GPUs = 2. Max memory: 79.205 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 9.0. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.69s/it]


此时8B模型所占显存如下：

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA H800 PCIe. Max memory = 79.205 GB.
7.865 GB of memory reserved.


然后进行问答测试：

In [ ]:
question_1 = "问题：某分裂组织在边境地区策划武装割据，并与境外势力秘密勾结，其首要分子甲在实施过程中被抓获。请问甲的行为构成何罪？应当如何量刑？"

In [ ]:
question_2 = "2015年9月4日14时50分许，被告人曹某某在景德镇市珠山区XX涵洞路口醉酒（血液酒精含量未明确数值）、无证驾驶无牌二轮摩托车，搭载李某林左转时未让直行车辆，与超速行驶的朱某良驾驶的出租车相撞，致李某林受伤（具体伤情等级未说明）。事故后曹某某赔偿李某林并获得谅解。公诉机关以危险驾驶罪指控，提供现场勘查记录、酒精检测等9项证据。本案中：1. 曹某某同时存在醉酒、无证、无牌三项违法情节，但缺乏具体酒精浓度数据；2. 事故系双方过错（曹某某未让行与朱某良超速）共同导致；3. 被害人已谅解。在此情况下：（1）危险驾驶罪的构成要件是否全部满足？（2）量刑时如何平衡\"无证+无牌+醉驾\"的从重情节与\"赔偿谅解\"的从轻情节？（3）最终判处3个月有期徒刑并处罚金1000元是否适当？"

In [ ]:
messages = [
    {"role" : "user", "content" : question_1}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    enable_thinking = True, # Disable thinking
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 20488, # Increase for longer outputs!
    temperature = 0.6, top_p = 0.95, top_k = 20, # For thinking
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

<think>
嗯，用户问的是甲在分裂组织策划武装割据，与境外势力勾结，被抓获后的行为构成什么罪，以及如何量刑。首先，我需要确定甲的行为属于什么犯罪。根据中国刑法，分裂国家、颠覆国家政权、煽动分裂国家都是罪名。但这里的情况是分裂组织，可能涉及分裂国家的犯罪，或者可能属于颠覆国家政权的犯罪？

首先，分裂国家罪和颠覆国家政权罪的区别。分裂国家罪是指国家分裂，比如分裂国家，但可能不涉及政权的颠覆，而颠覆国家政权罪则是指国家政权被推翻，比如通过武装叛乱、武装暴动等行为。甲是策划武装割据，可能涉及武装叛乱，但具体要看是否达到颠覆国家政权的程度。

另外，甲与境外势力勾结，可能涉及国际犯罪，比如间谍罪或者叛国罪？但根据中国刑法，分裂国家罪和颠覆国家政权罪可能有交叉。例如，如果甲的分裂行为导致国家分裂，可能构成分裂国家罪，同时可能涉及颠覆国家政权罪。

需要分析甲的行为是否构成这两个罪名中的一个或两个。比如，如果甲策划武装割据，可能属于颠覆国家政权的犯罪，因为武装割据可能构成颠覆国家政权的行为。同时，可能还有分裂国家罪，但可能两者合并处理。

另外，是否需要考虑甲是否有具体的实施行为，比如是否组织、策划、指挥等，这会影响罪名的轻重。例如，如果甲是首要分子，可能属于主犯，可能面临更重的刑罚。

量刑方面，根据刑法规定，分裂国家罪的刑罚是十年以上有期徒刑、无期徒刑或死刑；颠覆国家政权罪的刑罚是十年以上有期徒刑、无期徒刑或死刑。如果同时触犯两个罪名，可能数罪并罚，或者根据情节加重处罚。

还需要考虑是否有其他情节，比如是否造成严重后果，是否属于情节严重，是否属于组织、领导、教唆等，这会影响量刑的幅度。

可能还需要查阅相关法律条文，比如《刑法》第103条、第105条、第108条等，确认具体罪名和量刑标准。

总结：甲的行为可能构成分裂国家罪或颠覆国家政权罪，具体需要根据具体情节判断。量刑上可能在十年以上有期徒刑至无期徒刑或死刑，具体取决于情节严重程度和是否有其他因素。
</think>

根据中国《刑法》相关规定，甲的行为可能构成**分裂国家罪**或**颠覆国家政权罪**，具体需结合其行为的具体情节判断。以下是具体分析：

---

### **一、罪名分析**
1. **分裂国家罪**  
   - **构成要件**：以暴力、威胁手段分裂国家，或通过组织、策划武装割据，导致国家分裂

In [ ]:
messages = [
    {"role" : "user", "content" : question_2}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    enable_thinking = True, # Disable thinking
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 20488, # Increase for longer outputs!
    temperature = 0.6, top_p = 0.95, top_k = 20, # For thinking
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

<think>
嗯，用户问的是关于2015年一个交通事故案件，涉及曹某某的危险驾驶罪指控，还有关于量刑的平衡问题。我需要先理清楚每个问题的要点。

首先，第一问是危险驾驶罪的构成要件是否全部满足。根据中国的法律，危险驾驶罪需要满足几个条件：1. 醉酒驾驶；2. 无证驾驶；3. 无牌车辆。曹某某有醉酒、无证、无牌这三个违法情节，但缺乏具体酒精浓度数据。所以，虽然有三个违法点，但酒精浓度没有数据，可能影响是否构成危险驾驶罪的要件。不过，危险驾驶罪的构成是否需要酒精浓度的明确数据呢？可能需要看具体法律条文，比如刑法第133条的规定，危险驾驶罪是否要求醉酒驾驶，还是只要在驾驶过程中有危险行为即可。比如，如果曹某某没有酒精检测数据，但有醉酒的主观故意，可能构成危险驾驶罪，但需要看是否有证据支持。不过，用户提到的证据中只有现场勘查记录和酒精检测等9项证据，但酒精浓度未明确，所以可能无法确定是否醉酒。这时候可能需要判断是否构成危险驾驶罪的要件。

第二问是量刑时如何平衡无证、无牌、醉驾的从重情节和赔偿谅解的从轻情节。无证和无牌属于严重违法行为，可能从重，但醉驾是否需要酒精浓度数据呢？如果缺乏数据，可能无法认定为醉驾，但可能被认定为醉酒驾驶，但证据不足。这时候可能需要考虑是否有其他证据支持，比如曹某某的陈述或者医疗记录？但用户没有提到这些，所以可能需要假设这些证据存在，或者根据现有信息来判断。如果曹某某确实醉酒，但酒精浓度未明确，可能被认定为醉驾，但需要看是否有其他证据。不过，如果缺乏数据，可能无法确认，所以从重情节可能不成立，或者需要根据具体情况判断。

第三问是判处3个月有期徒刑并处罚金1000元是否适当。根据刑法，危险驾驶罪的量刑可能在拘役1-6个月，罚金500-2000元。3个月可能在范围内，但需要看是否有从轻情节。如果曹某某赔偿并谅解，可能减轻处罚，但需要看是否有其他因素。如果无证和无牌属于从重，但赔偿谅解从轻，可能综合考虑，但3个月是否过轻？或者是否在合理范围内？

可能需要查阅相关法律条文，比如刑法第133条，危险驾驶罪的构成要件，以及量刑标准。另外，关于醉驾是否需要酒精浓度数据，可能根据《关于办理醉酒驾驶机动车刑事案件适用法律若干问题的解释》的规定，醉酒驾驶需要血液酒精含量达到80mg/100ml以上，但用户提到的证据中没有明确数值，所以可能无法认定为醉驾，但可能被

### Step 2.数据集准备

&emsp;&emsp;接下来进行数据集读取与导入。这里除了要导入既定的高质量的法律问答数据集，同时也需要准备普通对话数据集。

In [ ]:
from datasets import Dataset
import json

In [ ]:
def load_jsonl_dataset(file_path):
    """从JSONL文件加载数据集"""
    data = {"input": [], "output": []}
    
    print(f"开始加载数据集: {file_path}")
    count = 0
    error_count = 0
    
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                item = json.loads(line.strip())
                # 根据数据集结构提取字段
                input_text = item.get("input", "")
                output = item.get("output", "")
                
                data["input"].append(input_text)
                data["output"].append(output)
                count += 1
            except Exception as e:
                print(f"解析行时出错: {e}")
                error_count += 1
                continue
    
    print(f"数据集加载完成: 成功加载{count}个样本, 跳过{error_count}个错误样本")
    return Dataset.from_dict(data)

In [ ]:
data_path = "./train_1k.jsonl"

In [ ]:
# 加载自定义数据集
dataset = load_jsonl_dataset(data_path)

# 显示数据集信息
print(f"\n数据集统计:")
print(f"- 样本数量: {len(dataset)}")
print(f"- 字段: {dataset.column_names}")

print(dataset[0])

开始加载数据集: ./train_1k.jsonl
数据集加载完成: 成功加载1000个样本, 跳过0个错误样本

数据集统计:
- 样本数量: 1000
- 字段: ['input', 'output']
{'input': '被告人周某被指控于2017年4月6日在绍兴市上虞区曹娥街道，通过虚构自己在蒿坝白鹤湾矿基石料场拥有20%股份的事实，并以该虚假股份作为抵押借款的手段，骗取被害人刘某1人民币8万元，所得赃款用于赌博、偿还债务及个人挥霍。根据《刑法》第266条诈骗罪的规定，周某的行为是否完全符合诈骗罪的构成要件？涉案金额8万元及赃款用途等情节，对28个月有期徒刑并处2万元罚金的量刑是否适当？', 'output': '<think>1. 犯罪构成要件分析\n   - 客观要件：\n     * 行为：周某实施了虚构股份持有事实（虚假陈述）→ 以虚假抵押借款（欺骗手段）→ 使被害人陷入错误认识并交付8万元（处分财产）→ 实际控制赃款后用于非法活动及个人消费（非法占有）\n     * 结果：被害人财产损失8万元，因果关系链条完整\n   - 主观要件：\n     * 直接故意：明知无股份仍虚构事实，具有非法占有目的（赃款用途印证赌博、还债等个人用途）\n\n2. 相关法条适用分析\n   - 《刑法》第266条诈骗罪要求"虚构事实+非法占有目的+财物转移"，本案完全符合：\n     * 虚构股份事实属于"虚构事实"\n     * 赌博挥霍用途证明"非法占有目的"\n     * 8万元属"数额较大"（浙江诈骗罪立案标准为6000元）\n\n3. 量刑情节分析\n   - 从重情节：\n     * 赃款用于违法犯罪活动（赌博）\n     * 无退赔情节（事实未提及退赃）\n   - 基准刑参照：\n     * 浙江诈骗6-10万基准刑2-3年，8万元对应28个月（2年4个月）符合幅度\n     * 罚金2万元为涉案金额25%（1-5倍范围内）\n\n4. 判决适当性评价\n   - 罪名认定准确：完全符合诈骗罪四要件\n   - 量刑适当：在法定幅度内，且考虑资金用途恶劣性\n   - 罚金比例合理：介于1-5倍标准之间</think>\n\n周某的行为完全符合《刑法》第266条诈骗罪的构成要件，其虚构股份抵押借款的行为具有明显的非法占有目的，且造成

In [ ]:
def formatting_prompts_func(examples):
    """根据提示模板格式化数据"""
    inputs = examples["input"]
    outputs = examples["output"]
    
    texts = []
    for  input_text, output in zip(inputs, outputs):
        texts.append([
            {"role" : "user",      "content" : input_text},
            {"role" : "assistant", "content" : output},
        ])
    
    return {"text": texts}

In [ ]:
# 应用格式化
print("开始格式化数据集...")
reasoning_conversations = tokenizer.apply_chat_template(
    dataset.map(formatting_prompts_func, batched = True)["text"],
    tokenize = False,
)
print("数据集格式化完成")

开始格式化数据集...


Map: 100%|██████████| 1000/1000 [00:00<00:00, 38004.28 examples/s]


数据集格式化完成


In [ ]:
reasoning_conversations[0]

'<|im_start|>user\n被告人周某被指控于2017年4月6日在绍兴市上虞区曹娥街道，通过虚构自己在蒿坝白鹤湾矿基石料场拥有20%股份的事实，并以该虚假股份作为抵押借款的手段，骗取被害人刘某1人民币8万元，所得赃款用于赌博、偿还债务及个人挥霍。根据《刑法》第266条诈骗罪的规定，周某的行为是否完全符合诈骗罪的构成要件？涉案金额8万元及赃款用途等情节，对28个月有期徒刑并处2万元罚金的量刑是否适当？<|im_end|>\n<|im_start|>assistant\n<think>\n1. 犯罪构成要件分析\n   - 客观要件：\n     * 行为：周某实施了虚构股份持有事实（虚假陈述）→ 以虚假抵押借款（欺骗手段）→ 使被害人陷入错误认识并交付8万元（处分财产）→ 实际控制赃款后用于非法活动及个人消费（非法占有）\n     * 结果：被害人财产损失8万元，因果关系链条完整\n   - 主观要件：\n     * 直接故意：明知无股份仍虚构事实，具有非法占有目的（赃款用途印证赌博、还债等个人用途）\n\n2. 相关法条适用分析\n   - 《刑法》第266条诈骗罪要求"虚构事实+非法占有目的+财物转移"，本案完全符合：\n     * 虚构股份事实属于"虚构事实"\n     * 赌博挥霍用途证明"非法占有目的"\n     * 8万元属"数额较大"（浙江诈骗罪立案标准为6000元）\n\n3. 量刑情节分析\n   - 从重情节：\n     * 赃款用于违法犯罪活动（赌博）\n     * 无退赔情节（事实未提及退赃）\n   - 基准刑参照：\n     * 浙江诈骗6-10万基准刑2-3年，8万元对应28个月（2年4个月）符合幅度\n     * 罚金2万元为涉案金额25%（1-5倍范围内）\n\n4. 判决适当性评价\n   - 罪名认定准确：完全符合诈骗罪四要件\n   - 量刑适当：在法定幅度内，且考虑资金用途恶劣性\n   - 罚金比例合理：介于1-5倍标准之间\n</think>\n\n周某的行为完全符合《刑法》第266条诈骗罪的构成要件，其虚构股份抵押借款的行为具有明显的非法占有目的，且造成被害人8万元的实际损失。法院判处28个月有期徒刑并处2万元罚金，符合浙江省关于诈骗罪"数额较大"的量刑标准，且考虑到赃款用于赌博等从重情节，该

In [ ]:
from datasets import load_dataset

In [ ]:
non_reasoning_dataset = load_dataset("mlabonne/FineTome-100k", split = "train")

Using the latest cached version of the dataset since mlabonne/FineTome-100k couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /root/.cache/huggingface/datasets/mlabonne___fine_tome-100k/default/0.0.0/c2343c1372ff31f51aa21248db18bffa3193efdb (last modified on Mon May  5 18:48:17 2025).


In [ ]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(non_reasoning_dataset)

non_reasoning_conversations = tokenizer.apply_chat_template(
    dataset["conversations"],
    tokenize = False,
)

In [ ]:
print(len(reasoning_conversations))
print(len(non_reasoning_conversations))

1000
100000


In [ ]:
import pandas as pd
non_reasoning_subset = pd.Series(non_reasoning_conversations)
non_reasoning_subset = non_reasoning_subset.sample(
    1000,
    random_state = 2407,
)

In [ ]:
print(len(reasoning_conversations))
print(len(non_reasoning_subset))

1000
1000


In [ ]:
data = pd.concat([
    pd.Series(reasoning_conversations),
    pd.Series(non_reasoning_subset)
])
data.name = "text"

In [ ]:
from datasets import Dataset
combined_dataset = Dataset.from_pandas(pd.DataFrame(data))
combined_dataset = combined_dataset.shuffle(seed = 3407)

In [ ]:
len(combined_dataset)

2000

In [ ]:
combined_dataset[3]

{'text': '<|im_start|>user\n被告人刘X于2014年2月15日凌晨2时许，在齐齐哈尔市龙沙区雅乐迪KTV歌厅内秘密窃取一楼大厅功德箱（内有人民币10000余元），后于2014年3月7日在佳木斯市被抓获。其归案后如实供述犯罪事实、自愿认罪并积极返还赃款。根据《中华人民共和国刑法》相关规定，刘X的行为是否构成盗窃罪？若构成犯罪，在涉案金额达"数额较大"标准且存在认罪悔罪情节的情况下，判处8个月有期徒刑并处10000元罚金的判决是否适当？<|im_end|>\n<|im_start|>assistant\n<think>\n1. **犯罪构成要件分析**  \n   - **客观要件**：  \n     - **行为**：问题明确刘X"秘密窃取"功德箱，符合盗窃罪"以非法占有为目的，秘密窃取公私财物"的行为特征。  \n     - **结果**：涉案金额10000元，根据《最高人民法院、最高人民检察院关于办理盗窃刑事案件适用法律若干问题的解释》，已达到"数额较大"标准（1000元至3000元以上）。  \n     - **因果关系**：刘X的窃取行为直接导致被害人财产损失。  \n   - **主观要件**：问题中"秘密窃取"的表述及盗走功德箱的行为，可推定其具有非法占有的直接故意。  \n\n2. **相关法条适用分析**  \n   - 根据《刑法》第264条，盗窃公私财物数额较大的，处3年以下有期徒刑、拘役或管制，并处或单处罚金。本案金额10000元符合该条款的"数额较大"标准。  \n   - 刘X"如实供述、自愿认罪、返还赃款"的情节，符合《刑法》第67条第3款（坦白从宽）及司法解释关于退赃从轻处罚的规定。  \n\n3. **量刑情节分析**  \n   - **从轻情节**：  \n     - 坦白认罪（《刑法》第67条第3款）；  \n     - 积极退赃（司法解释规定可减少基准刑30%以下）。  \n   - **无从重情节**：问题未提及累犯、暴力手段等加重情节。  \n\n4. **判决适当性评价**  \n   - 基准刑：盗窃数额较大的法定刑为3年以下，司法实践中类似案件基准刑通常为6-12个月。  \n   - 调整幅度：坦白+退赃可减少基准刑40%以下，实际判处8个月（接近基准刑下限）符合量刑规范化要求。 

In [ ]:
combined_dataset[10]

{'text': '<|im_start|>user\nWrite a Python program that implements a basic calculator with support for addition, subtraction, multiplication, and division operations. The calculator should accept user input in the form of two numbers and an operator symbol (+, -, *, /) and display the result.<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\nHere\'s a simple Python program that implements a basic calculator with support for addition, subtraction, multiplication, and division operations:\n\n```python\ndef add(x, y):\n    return x + y\n\ndef subtract(x, y):\n    return x - y\n\ndef multiply(x, y):\n    return x * y\n\ndef divide(x, y):\n    if y == 0:\n        raise ValueError("Cannot divide by zero")\n    return x / y\n\ndef main():\n    print("Select operation:")\n    print("1. Add")\n    print("2. Subtract")\n    print("3. Multiply")\n    print("4. Divide")\n\n    choice = input("Enter the number of the operation you want to perform (1/2/3/4): ")\n\n    num1 = float(input("Ente

### Step 3.执行微调流程

- 进行LoRA参数注入

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,           # Choose any number > 0! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,  # Best to choose alpha = rank or rank*2
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,   # We support rank stabilized LoRA
    loftq_config = None,  # And LoftQ
)

Unsloth 2025.4.7 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


- 设置微调参数

In [ ]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = combined_dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = None, # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=176): 100%|██████████| 2000/2000 [00:53<00:00, 37.15 examples/s]


此时显存占用如下：

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA H800 PCIe. Max memory = 79.205 GB.
7.865 GB of memory reserved.


- 微调执行流程

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,000 | Num Epochs = 1 | Total steps = 30
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 87,293,952/8,000,000,000 (1.09% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: 2323365771 (2323365771-ff) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.662500
2,1.652300
3,1.813100
4,1.501600
5,1.565300
6,1.466600
7,1.311800
8,1.248200
9,0.987500
10,1.152200


- 微调期间显存占用检测

In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

101.5196 seconds used for training.
1.69 minutes used for training.
Peak reserved memory = 10.135 GB.
Peak reserved memory for training = 2.27 GB.
Peak reserved memory % of max memory = 12.796 %.
Peak reserved memory for training % of max memory = 2.866 %.


最后是问答效果测试：

In [ ]:
question_1 = "问题：某分裂组织在边境地区策划武装割据，并与境外势力秘密勾结，其首要分子甲在实施过程中被抓获。请问甲的行为构成何罪？应当如何量刑？"

In [ ]:
question_2 = "2015年9月4日14时50分许，被告人曹某某在景德镇市珠山区XX涵洞路口醉酒（血液酒精含量未明确数值）、无证驾驶无牌二轮摩托车，搭载李某林左转时未让直行车辆，与超速行驶的朱某良驾驶的出租车相撞，致李某林受伤（具体伤情等级未说明）。事故后曹某某赔偿李某林并获得谅解。公诉机关以危险驾驶罪指控，提供现场勘查记录、酒精检测等9项证据。本案中：1. 曹某某同时存在醉酒、无证、无牌三项违法情节，但缺乏具体酒精浓度数据；2. 事故系双方过错（曹某某未让行与朱某良超速）共同导致；3. 被害人已谅解。在此情况下：（1）危险驾驶罪的构成要件是否全部满足？（2）量刑时如何平衡\"无证+无牌+醉驾\"的从重情节与\"赔偿谅解\"的从轻情节？（3）最终判处3个月有期徒刑并处罚金1000元是否适当？"

In [ ]:
messages = [
    {"role" : "user", "content" : question_1}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    enable_thinking = True, # Disable thinking
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 20488, # Increase for longer outputs!
    temperature = 0.6, top_p = 0.95, top_k = 20, # For thinking
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

<think>
嗯，用户问的是关于分裂组织策划武装割据，与境外势力勾结，首要分子甲被抓获，问甲的行为构成何罪，如何量刑。首先，我需要确定中国刑法中相关的罪名。根据中国法律，分裂国家、破坏国家统一属于严重犯罪，可能涉及分裂国家罪、煽动分裂国家罪，还有可能涉及恐怖活动相关的罪名。

首先，分裂国家罪和煽动分裂国家罪是《刑法》第103条规定的，如果组织、策划、实施分裂国家、破坏国家统一的行为，或者煽动他人从事这些行为，会构成相关罪名。另外，如果涉及境外势力，可能还涉及“勾结境外势力”或者“资助恐怖活动组织”等罪名。

然后，根据《刑法》第103条，分裂国家罪的刑罚是三年以上十年以下有期徒刑，如果情节特别严重，可以处十年以上有期徒刑或者无期徒刑。而煽动分裂国家罪的刑罚是三年以下有期徒刑、拘役、管制或者剥夺政治权利，如果情节严重的，可以处三年以上七年以下有期徒刑。不过，这里甲是首要分子，可能属于组织、领导、参加恐怖活动组织或者恐怖活动集团的，可能适用更重的刑罚。

另外，如果甲的行为涉及与境外势力勾结，可能还构成“勾结境外机构、组织、个人危害中华人民共和国主权、领土完整和安全罪”，根据《刑法》第113条，这种情况下，刑罚可以是十年以上有期徒刑、无期徒刑或者死刑。同时，如果甲的行为属于恐怖活动，可能涉及《刑法》第120条的组织、领导、参加恐怖组织罪，或者第120条之一的资助恐怖活动罪，这些罪名的刑罚通常较重，比如组织、领导恐怖组织的，处三年以上十年以下有期徒刑，情节严重的，处十年以上有期徒刑或者无该。

不过，这里的关键是甲的行为是否属于恐怖活动，或者是否属于分裂国家的犯罪。如果甲的行为是策划武装割据，可能更倾向于分裂国家罪，而如果涉及与境外势力勾结，可能同时构成勾结境外势力罪。此外，是否属于“恐怖活动”需要具体分析，比如是否有使用暴力、威胁等手段，是否针对国家政权等。

另外，根据《刑法》第103条，对于分裂国家罪，如果行为人是首要分子，可能被认定为“情节特别严重”，从而适用更重的刑罚。同时，如果甲在实施过程中有其他严重情节，比如造成重大损失、危害公共安全等，也可能影响量刑。

需要注意的是，中国刑法对于分裂国家、破坏国家统一的犯罪有严格的法律规定，且刑罚较重。此外，与境外势力勾结的行为，如果构成犯罪，会进一步加重刑罚。因此，甲可能同时触犯多个罪名，需要根据具体行为进行数罪并

In [ ]:
messages = [
    {"role" : "user", "content" : question_2}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    enable_thinking = True, # Disable thinking
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 20488, # Increase for longer outputs!
    temperature = 0.6, top_p = 0.95, top_k = 20, # For thinking
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

<think>
好的，我现在需要分析这个关于曹某某危险驾驶罪的案例。首先，我得回忆一下危险驾驶罪的构成要件，然后看看本案中是否满足这些要件。然后，关于量刑的问题，需要考虑从重和从轻的情节，以及最终判决是否合理。让我一步步来思考。

首先，危险驾驶罪的构成要件。根据中国刑法第133条之一，危险驾驶罪包括醉酒驾驶机动车、超速驾驶、追逐竞驶、严重超载等行为。这里曹某某的情况是醉酒驾驶，无证驾驶，无牌摩托车，同时发生事故，导致他人受伤。但题目中提到血液酒精含量未明确数值，所以可能需要考虑是否达到醉驾的标准。

危险驾驶罪的构成要件通常包括：1. 主观上具有故意，即明知醉酒驾驶；2. 客观上实施了醉酒驾驶机动车的行为；3. 行为具有危险性，可能危害公共安全。这里曹某某醉酒驾驶，即使没有酒精浓度数据，但根据案件描述，他确实醉酒了，所以可能被认定为醉驾。但如果没有具体数值，可能需要进一步证据，比如现场检测结果，或者是否有其他证据表明他处于醉酒状态。比如，现场勘查记录、酒精检测等证据，可能已经足够证明醉驾，即使数值未明确，但可能有检测报告，或者目击者证词等。

接下来，第二个问题，量刑时如何平衡从重和从轻情节。曹某某有三个违法情节：醉驾、无证、无牌，这些通常会被视为从重情节。但被害人已经谅解，且曹某某赔偿了损失，这可能属于从轻情节。需要考虑这些情节的权重，以及是否符合法律规定。

第三个问题，是否判处3个月有期徒刑并处罚金1000元是否适当。这需要根据量刑指导意见，比如危险驾驶罪的量刑起点，以及从重、从轻情节的调整。比如，如果醉驾且造成事故，可能在量刑上会更重，但如果有赔偿和谅解，可能减轻。

现在，我需要更详细地分析每个部分。

首先，危险驾驶罪的构成要件是否满足。根据刑法，醉酒驾驶属于危险驾驶罪，但需要确认是否达到醉驾的标准。虽然题目中说血液酒精含量未明确数值，但可能有其他证据，比如现场检测报告，或者目击者描述，比如曹某某明显醉酒，所以可能被认定为醉驾。因此，危险驾驶罪的构成要件应该满足，因为醉驾是构成要件之一，而其他情节如无证、无牌可能属于加重情节，但并不影响构成要件的满足。

第二个问题，量刑平衡。根据最高人民法院的量刑指导意见，对于醉驾，如果造成交通事故，且负事故责任，可能会从重处罚。但如果有赔偿和谅解，可能从轻。此外，无证、无牌属于从重情节，但赔偿谅解属于从轻情节。需要

能够看出问答语气风格有明显优化，表述更加完整，答案也更加专业。